# O-information

## Index

1. [Functions](#functions)
1. [Load and binarize data](#load_binar_data)
    1. [Gaussian randomization](#load_binar_data_gaussian)
        1. [Goodness of the null model](#load_binar_data_gaussian_goodness)
1. [O-information](#o_info)
    1. [Full length](#o_info_full_length)
        1. [Probability density function](#o_info_full_length_pdf)
            1. [Friedman test](#o_info_full_length_pdf_friedman)
        1. [Null model](#o_info_full_length_null_model)
            1. [Statistical significance](#o_info_full_length_null_model_stat)
                1. [t-test](#o_info_full_length_null_model_stat_t_test)
                1. [Z-score](#o_info_full_length_null_model_stat_Z_score)
                1. [Comparison](#o_info_full_length_null_model_stat_comparison)
        1. [Null model no multivariate](#o_info_full_length_null_model_no_multivariate)
    1. [Divided by year](#o_info_divided_year)
        1. [Null model](#o_info_one_year_null_model)
            1. [Randomised and divided by year](#o_info_one_year_null_model_randomised_divided)
                1. [Statistical significance](#o_info_one_year_null_model_randomised_divided_stat)
                    1. [t-test](#o_info_one_year_null_model_randomised_divided_stat_t_test)
                    1. [Z-score](#o_info_one_year_null_model_randomised_divided_stat_Z_score)
                    1. [Comparison](#o_info_one_year_null_model_randomised_divided_stat_comparison)
            1. [Divided by year and randomised](#o_info_one_year_null_model_divided_randomised)
                1. [Statistical significance](#o_info_one_year_null_model_divided_randomised_stat)
                    1. [t-test](#o_info_one_year_null_model_divided_randomised_stat_t_test)
                    1. [Z-score](#o_info_one_year_null_model_divided_randomised_stat_Z_score)
                    1. [Comparison](#o_info_one_year_null_model_divided_randomised_stat_comparison)
        1. [Evolution in time](#o_info_one_year_evolution_time)

<a name="functions"></a>
## Functions

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.patches as mpatches
from itertools import combinations, product
from tqdm.auto import tqdm
import pickle
from scipy.special import kl_div
from scipy.stats import entropy, friedmanchisquare, norm
from sklearn.utils import resample, check_random_state
from scipy import stats
import random
import os
from statsmodels.stats.multitest import multipletests

from ho_info_metrics.metrics import *
from ho_info_metrics.utils import *

In [ ]:
def binarize_df(df, window_size):
    df = df.drop(columns=["Date"])
    for col in df.columns:
        if col != "Date":
            col_values = df[col].values
            binarized_values = []
            for i in range(len(col_values)):
                window_start = max(0, i - window_size + 1)
                window_mean = col_values[window_start : i + 1].mean()
                binarized_values.append(1 if col_values[i] > window_mean else 0)
            df[col] = binarized_values
    return df

In [ ]:
def get_o_infos_same(target_sector, binarized_ts, total=False):
    """Get the O-information for the triplets of the target sector"""

    print(">>>", target_sector)

    target_stocks = list(
        stock_names[stock_names["Sector"] == target_sector]["Stock name"]
    )  # tutti gli stock name il cui settore sia il target
    print("Length target_stocks:", len(target_stocks))

    if total:
        same_sector_triplets = list(combinations(target_stocks, 3))
    else:
        # Having removed the Technology sector, each sector can now offer 35 triplets
        same_sector_triplets = list(combinations(target_stocks, 3))[:35]
    print("Length triplets same sector:", len(same_sector_triplets))

    oinfo_one_sector = []
    bar_length = len(same_sector_triplets)
    with tqdm(total=bar_length) as pbar:
        pbar.set_description("Same sector")
        for i in range(len(same_sector_triplets)):
            X1 = binarized_ts[same_sector_triplets[i][0]].values
            X2 = binarized_ts[same_sector_triplets[i][1]].values
            X3 = binarized_ts[same_sector_triplets[i][2]].values
            X = np.vstack((X1, X2, X3))
            oinfo_one_sector.append(
                o_information_boot(X, estimator="cat_ent")
            )  # calcola la O-information della tripletta
            pbar.update(1)

    return oinfo_one_sector

In [ ]:
def get_o_infos_two_different(target_sector, another_sector, binarized_ts, total=False):
    """Get the O-information for the triplets composed of one
    node in the target_sector and two in the another_sector"""

    print(">>>", target_sector, "-", another_sector)

    target_stocks = list(
        stock_names[stock_names["Sector"] == target_sector]["Stock name"]
    )
    another_stocks = list(
        stock_names[stock_names["Sector"] == another_sector]["Stock name"]
    )
    print("Length another_stocks:", len(another_stocks))

    if total:
        another_sector_triplets = []
        for target in target_stocks:
            for combo in combinations(another_stocks, 2):
                another_sector_triplets.append((target, combo[0], combo[1]))
        print("Length triplets two sectors:", len(another_sector_triplets))

    else:
        # Having removed the Technology sector, each sector can now offer 35 triplets
        another_sector_triplets = []
        while len(another_sector_triplets) != 35:
            triplet = []
            triplet.append(np.random.choice(target_stocks))
            triplet.append(np.random.choice(another_stocks))
            triplet.append(np.random.choice(another_stocks))
            another_sector_triplets.append(triplet)
        print("Length triplets two sectors:", len(another_sector_triplets))

    oinfo_two_sectors = []
    bar_length = len(another_sector_triplets)
    with tqdm(total=bar_length) as pbar:
        pbar.set_description("Two sectors")
        for i in range(len(another_sector_triplets)):
            X1 = binarized_ts[another_sector_triplets[i][0]].values
            X2 = binarized_ts[another_sector_triplets[i][1]].values
            X3 = binarized_ts[another_sector_triplets[i][2]].values
            X = np.vstack((X1, X2, X3))
            oinfo_two_sectors.append(o_information_boot(X, estimator="cat_ent"))
            pbar.update(1)

    return oinfo_two_sectors

In [ ]:
def get_o_infos_three_different(
    target_sector, sector1, sector2, binarized_ts, total=False
):
    """Get the O-information for the triplets
    composed of three nodes in three different sectors"""

    print(">>>", target_sector, "-", sector1, "-", sector2)

    target_stocks = list(
        stock_names[stock_names["Sector"] == target_sector]["Stock name"]
    )
    stocks1 = list(stock_names[stock_names["Sector"] == sector1]["Stock name"])
    print("Length sector1:", len(stocks1))
    stocks2 = list(stock_names[stock_names["Sector"] == sector2]["Stock name"])
    print("Length sector2:", len(stocks2))

    if total:
        three_sectors_triplets = []
        for combo in product(target_stocks, stocks1, stocks2):
            three_sectors_triplets.append(combo)
        print("Length triplets three sectors:", len(three_sectors_triplets))

    else:
        # Having removed the Technology sector, each sector can now offer 35 triplets
        three_sectors_triplets = []
        while len(three_sectors_triplets) != 35:
            triplet = []
            triplet.append(np.random.choice(target_stocks))
            triplet.append(np.random.choice(stocks1))
            triplet.append(np.random.choice(stocks2))
            three_sectors_triplets.append(triplet)
        print("Length triplets three sectors:", len(three_sectors_triplets))

    oinfo_three_sectors = []
    bar_length = len(three_sectors_triplets)
    with tqdm(total=bar_length) as pbar:
        pbar.set_description("Three sectors")
        for i in range(len(three_sectors_triplets)):
            X1 = binarized_ts[three_sectors_triplets[i][0]].values
            X2 = binarized_ts[three_sectors_triplets[i][1]].values
            X3 = binarized_ts[three_sectors_triplets[i][2]].values
            X = np.vstack((X1, X2, X3))
            oinfo_three_sectors.append(o_information_boot(X, estimator="cat_ent"))
            pbar.update(1)

    return oinfo_three_sectors

In [ ]:
def t_bootstrap(sample1, sample2, n_bootstrap_samples=1000):
    """Compute the t-test after having done the bootstrap and return the t-value and p-value 
    (with Bonferroni correction)"""

    # Stores the differences between the sample means
    mean_differences = np.zeros(n_bootstrap_samples)

    for i in range(n_bootstrap_samples):
        # Sampling with replacement
        resampled_sample1 = resample(sample1, random_state=check_random_state(i))
        resampled_sample2 = resample(sample2, random_state=check_random_state(i))

        # Calculate the average of each resampled sample
        resampled_mean1 = np.mean(resampled_sample1)
        resampled_mean2 = np.mean(resampled_sample2)

        # Calculates the difference between the resampled sample means
        mean_differences[i] = resampled_mean1 - resampled_mean2

    # Calculate the 95% confidence interval for the difference in the sample means
    confidence_interval = np.percentile(mean_differences, [2.5, 97.5])

    # Calculate the observed t-value
    observed_diff = np.mean(sample1) - np.mean(sample2)
    se = np.std(mean_differences)
    t_value = observed_diff / se

    # Calculate the p-value
    p_value = 2 * (1 - stats.norm.cdf(abs(t_value)))

    # Apply the Bonferroni correction
    _, p_adjusted, _, _ = multipletests(
        p_value, alpha=0.05, method="bonferroni", is_sorted=False, returnsorted=False
    )

    return t_value, p_adjusted

In [ ]:
def t_test(sample1, sample2):
    """Compute the t-test of the two samples"""
    t_statistic, p_value = stats.ttest_ind(sample1, sample2)

    return t_statistic, p_value

In [ ]:
def sign_t(x):
    return "+" if x >= 0 else "-"


def sign_p(x):
    return "red" if x >= 0.05 else "green"

In [ ]:
def z_score(oinfo_original, oinfo_gauss):
    """Compute the Z-score and p-value"""

    mean_oinfo_original = np.mean(oinfo_original)
    mean_oinfo_gauss = np.mean(oinfo_gauss)
    std_oinfo_gauss = np.std(oinfo_gauss)

    Z = (mean_oinfo_original - mean_oinfo_gauss) / std_oinfo_gauss
    p_value = 2 * (1 - norm.cdf(abs(Z)))

    return Z, p_value

<a name="load_binar_data"></a>
## Load and binarize data

In [ ]:
financial_ts = pd.read_csv(
    "data/Economic_data/NYSE_119stocks_2000Jan_2021June_withdates.csv"
)
stock_names = pd.read_csv(
    "data/Economic_data/list_stocknames.txt",
    sep=", ",
    names=["Index", "Stock name", "Sector"],
)

In [ ]:
financial_ts

In [ ]:
stock_names

In [ ]:
tw = 7  # time window, originalmente era 2
binarized_financial_ts = binarize_df(financial_ts, tw)

In [ ]:
binarized_financial_ts

In [ ]:
sectors = list(stock_names["Sector"].unique())
sectors

In [ ]:
for sector in sectors:
    target_stocks = list(stock_names[stock_names["Sector"] == sector]["Stock name"])
    same_sector_triplets = list(combinations(target_stocks, 3))
    print(f"Total number of triplets in {sector}: {len(same_sector_triplets)}")

<a name="load_binar_data_gaussian"></a>
### Gaussian randomization

In [ ]:
media_colonne = financial_ts.iloc[:, 1:].mean()
covarianza = financial_ts.iloc[:, 1:].cov()

In [ ]:
num_campioni = len(financial_ts)
valori_generati = np.random.multivariate_normal(media_colonne, covarianza, num_campioni)

In [ ]:
financial_ts_gauss = pd.DataFrame(
    valori_generati, columns=financial_ts.iloc[:, 1:].columns
)
financial_ts_gauss.insert(
    0, financial_ts.columns[0], financial_ts[financial_ts.columns[0]]
)

In [ ]:
financial_ts_gauss

In [ ]:
tw = 7  # time window, originalmente era 2
binarized_financial_ts_gauss = binarize_df(financial_ts_gauss, tw)

In [ ]:
binarized_financial_ts_gauss

<a name="load_binar_data_gaussian_goodness"></a>
#### Goodness of the null model

In [ ]:
covariance_original = financial_ts.iloc[:, 1:].cov()
covariance_gauss = financial_ts_gauss.iloc[:, 1:].cov()

In [ ]:
threshold = 0.05
count, total_count = 0, 0
relative_difference = []

for column in covariance_original.columns:
    for row in covariance_original.columns:
        total_count += 1
        difference = covariance_original[column][row] - covariance_gauss[column][row]
        relative_difference.append(abs(difference / covariance_original[column][row]))
        if relative_difference[total_count - 1] > threshold:
            count += 1
relative_difference.sort()

print(
    f"Number of points exceeding the threshold: {count} out of {total_count} ({round(count / total_count, 3) * 100} %)"
)

In [ ]:
plt.figure(figsize=(8, 6))

average = np.mean(relative_difference)

plt.hist(relative_difference[:-10], bins=20, edgecolor="black")
plt.axvline(average, color="r", linestyle="dashed", linewidth=1.7)

plt.yscale("log")
plt.xlabel("Relative difference", size=15)
plt.ylabel("Frequency", size=15)

plt.text(
    0.95,
    0.95,
    f"For clarity I have removed\nthe last 10 values; the\nmaximum value was {int(max(relative_difference))}",
    transform=plt.gca().transAxes,
    verticalalignment="top",
    horizontalalignment="right",
    fontsize=7,
)
plt.text(0.95, 0.85, f"mean = {round(average, 3)}",
    transform=plt.gca().transAxes,
    verticalalignment="top",
    horizontalalignment="right",
    fontsize=7,)

plt.savefig(
    f"./results/economic_results/o_info/images/all/gaussian/goodness_null_model_relative_difference_covariance.pdf",
    dpi=600,
    bbox_inches="tight",
)

plt.show()

<a name="o_info"></a>
## O-information

In [ ]:
sectors = [
    "Industrials",
    "Basic Materials",
    "Financials",
    "Consumer Services",
    "Health Care",
    "Utilities",
    "Oil & Gas",
    "Consumer Goods",
]

In [ ]:
# I am deleting the data for the Techonolgy sector because there are only a few
financial_ts = financial_ts.drop(columns=["CACI", "CIEN", "DBD", "DDD"])
financial_ts_gauss = financial_ts_gauss.drop(columns=["CACI", "CIEN", "DBD", "DDD"])

In [ ]:
binarized_financial_ts = binarized_financial_ts.drop(
    columns=["CACI", "CIEN", "DBD", "DDD"]
)
binarized_financial_ts_gauss = binarized_financial_ts_gauss.drop(
    columns=["CACI", "CIEN", "DBD", "DDD"]
)

<a name="o_info_full_length"></a>
### Full length

In [ ]:
# DO NOT RUN IF YOU ALREADY HAVE THE DATA

for target in sectors:
    oinfo_one_sector = get_o_infos_same(target, binarized_financial_ts, total=True)
    np.save(
        f"./results/economic_results/o_info/metrics/3/all/one_sector_{target}.npy",
        oinfo_one_sector,
    )

for target in sectors:
    other_sectors = [sector for sector in sectors if sector != target]
    for i, sector in enumerate(other_sectors):

        oinfo_two_sectors = get_o_infos_two_different(
            target, sector, binarized_financial_ts, total=True
        )
        np.save(
            f"./results/economic_results/o_info/metrics/2_1/all/one_sector_{target}_two_sector_{sector}.npy",
            oinfo_two_sectors,
        )

for target in sectors:
    other_sectors = [sector for sector in sectors if sector != target]
    for sector1 in other_sectors:
        other_sectors2 = [sector for sector in other_sectors if sector != sector1]
        for sector2 in other_sectors2:

            oinfo_three_sectors = get_o_infos_three_different(
                target, sector1, sector2, binarized_financial_ts, total=True
            )
            np.save(
                f"./results/economic_results/o_info/metrics/1_1_1/all/one_sector_{target}_one_sector_{sector1}_one_sector_{sector2}.npy",
                oinfo_three_sectors,
            )

In [ ]:
# DO NOT RUN IF YOU ALREADY HAVE THE DATAFRAMES

for target1 in sectors:
    df_matrix = pd.DataFrame(columns=sectors)
    for target2 in sectors:
        for source in sectors:

            if target2 == target1 and source == target1:
                oinfo_one_sector = np.load(
                    f"./results/economic_results/o_info/metrics/3/all/one_sector_{target2}.npy"
                )
                df_matrix.at[target2, source] = np.mean(oinfo_one_sector)

            elif target2 == target1 and source != target1:
                oinfo_two_sectors = np.load(
                    f"./results/economic_results/o_info/metrics/2_1/all/one_sector_{source}_two_sector_{target2}.npy"
                )
                df_matrix.at[target2, source] = np.mean(oinfo_two_sectors)

            elif target2 != target1 and source == target1:
                oinfo_two_sectors = np.load(
                    f"./results/economic_results/o_info/metrics/2_1/all/one_sector_{target2}_two_sector_{source}.npy"
                )
                df_matrix.at[target2, source] = np.mean(oinfo_two_sectors)

            elif target2 != target1 and source == target2:
                oinfo_two_sectors = np.load(
                    f"./results/economic_results/o_info/metrics/2_1/all/one_sector_{target1}_two_sector_{target2}.npy"
                )
                df_matrix.at[target2, source] = np.mean(oinfo_two_sectors)

            elif target2 != target1 and source != target2:
                oinfo_three_sectors = np.load(
                    f"./results/economic_results/o_info/metrics/1_1_1/all/one_sector_{target1}_one_sector_{target2}_one_sector_{source}.npy"
                )
                df_matrix.at[target2, source] = np.mean(oinfo_three_sectors)

    df_matrix.to_csv(
        f"./results/economic_results/o_info/dataframes/all/matrix_{target1}.csv",
        index=True,
    )

In [ ]:
for sector in sectors:

    df = pd.read_csv(
        f"./results/economic_results/o_info/dataframes/all/matrix_{sector}.csv",
        index_col=0,
    )
    sns.heatmap(df, cmap="Blues")

    plt.xlabel("Source", fontsize=13)
    plt.ylabel("Target", fontsize=13)
    plt.title(f"1 in {sector}, 1 in Target, 1 in Source", fontsize=20)

    plt.savefig(
        f"./results/economic_results/o_info/images/all/heatmap_matrix_{sector}.pdf",
        dpi=600,
        bbox_inches="tight",
    )

    plt.show()

In [ ]:
mini, maxi = 50, -50
for sector in sectors:
    df = pd.read_csv(
        f"./results/economic_results/o_info/dataframes/all/matrix_{sector}.csv",
        index_col=0,
    )
    if min(df.min()) < mini:
        mini = min(df.min())
    if max(df.max()) > maxi:
        maxi = max(df.max())

fig, axs = plt.subplots(2, 4, figsize=(20, 10))

for index, sector in enumerate(sectors):
    row = index // 4
    col = index % 4

    ax = axs[row, col]

    df = pd.read_csv(
        f"./results/economic_results/o_info/dataframes/all/matrix_{sector}.csv",
        index_col=0,
    )
    sns.heatmap(df, ax=ax, cmap="Blues", vmin=mini, vmax=maxi)

    ax.set_title(f"{sector}", size=19)

    ax.set_xticklabels([])
    ax.set_yticklabels([])

fig.text(0.49, -0.02, "Source", ha="center", va="center", fontsize=24)
fig.text(-0.02, 0.49, "Target", rotation="vertical", va="center", fontsize=24)

plt.tight_layout()

plt.savefig(
    f"./results/economic_results/o_info/images/all/heatmap_matrix_all_sectors.pdf",
    dpi=600,
    bbox_inches="tight",
)

plt.show()

<a name="o_info_full_length_pdf"></a>
#### Probability density function

In [ ]:
# DO NOT RUN IF YOU ALREADY HAVE THE DICTIONARY

diz_pdf = {
    "Industrials": None,
    "Basic Materials": None,
    "Financials": None,
    "Consumer Services": None,
    "Health Care": None,
    "Utilities": None,
    "Oil & Gas": None,
    "Consumer Goods": None,
}

for target in sectors:
    diz_sector = {"3": None, "2_1": None, "1_1_1": None}

    for key in diz_sector.keys():
        if key == "3":
            diz_sector[key] = list(
                np.load(
                    f"./results/economic_results/o_info/metrics/3/all/one_sector_{target}.npy"
                )
            )

        elif key == "2_1":
            a = list()
            other_sectors = [sector for sector in sectors if sector != target]
            for source in other_sectors:
                b = list(
                    np.load(
                        f"./results/economic_results/o_info/metrics/2_1/all/one_sector_{source}_two_sector_{target}.npy"
                    )
                )
                a.append(b)
            diz_sector[key] = [item for sublist in a for item in sublist]

        elif key == "1_1_1":
            a = list()
            other_sectors1 = [sector for sector in sectors if sector != target]
            for source1 in other_sectors1:
                other_sectors2 = [
                    sector for sector in other_sectors1 if sector != source1
                ]
                for source2 in other_sectors2:
                    b = list(
                        np.load(
                            f"./results/economic_results/o_info/metrics/1_1_1/all/one_sector_{target}_one_sector_{source1}_one_sector_{source2}.npy"
                        )
                    )
                    a.append(b)
            diz_sector[key] = [item for sublist in a for item in sublist]

    diz_pdf[target] = diz_sector

with open(
    "./results/economic_results/o_info/dataframes/all/dict_proba_density_function_all.pickle",
    "wb",
) as file:
    pickle.dump(diz_pdf, file)

In [ ]:
with open(
    "./results/economic_results/o_info/dataframes/all/dict_proba_density_function_all.pickle",
    "rb",
) as file:
    diz_pdf = pickle.load(file)

In [ ]:
for sector in sectors:
    keys = list(diz_pdf[sector].keys())
    data = [diz_pdf[sector][key] for key in keys]
    means = [np.mean(data[i]) for i in range(len(data))]

    plt.figure(figsize=(8, 6))
    for i, key in enumerate(keys):
        values_bin, bin_edges = np.histogram(data[i], bins=20)
        if i == 0:
            label = str(f"3 in {sector}")
        elif i == 1:
            label = str(f"2 in {sector}")
        else:
            label = str(f"1 in {sector}")

        plt.bar(
            bin_edges[:-1],
            values_bin / sum(values_bin),
            width=np.diff(bin_edges),
            align="edge",
            alpha=0.6,
            label=label,
        )

    plt.axvline(x=means[0], color="blue", linestyle="--")
    plt.axvline(x=means[1], color="orange", linestyle="--")
    plt.axvline(x=means[2], color="green", linestyle="--")

    plt.xlabel("O-information", size=15)
    plt.ylabel("Probability", size=15)
    plt.tick_params(axis="both", labelsize=12)
    plt.title(f"PDF of {sector}", size=18)
    plt.legend(fontsize="large")

    plt.savefig(
        f"./results/economic_results/o_info/images/all/pdf/PDF_{sector}.pdf",
        dpi=600,
        bbox_inches="tight",
    )

    plt.show()

In [ ]:
fig, axs = plt.subplots(2, 4, figsize=(15, 8))

for index, sector in enumerate(sectors):
    keys = list(diz_pdf[sector].keys())
    data = [diz_pdf[sector][key] for key in keys]
    means = [np.mean(data[i]) for i in range(len(data))]

    row = index // 4
    col = index % 4

    ax = axs[row, col]

    for i, key in enumerate(keys):
        values_bin, bin_edge = np.histogram(data[i], bins=20)

        if i == 0:
            label = str(f"3 in {sector}")
        elif i == 1:
            label = str(f"2 in {sector}")
        else:
            label = str(f"1 in {sector}")

        ax.bar(
            bin_edge[:-1],
            values_bin / sum(values_bin),
            width=np.diff(bin_edge),
            align="edge",
            alpha=0.6,
            label=label,
        )

    ax.axvline(x=means[0], color="blue", linestyle="--")
    ax.axvline(x=means[1], color="orange", linestyle="--")
    ax.axvline(x=means[2], color="green", linestyle="--")

    ax.set_title(f"PDF of {sector}", size=17)
    ax.legend(fontsize="small")

fig.text(0.5, -0.02, "O-information", ha="center", va="center", fontsize=22)
fig.text(-0.015, 0.5, "Probability", rotation="vertical", va="center", fontsize=22)

plt.tight_layout()

plt.savefig(
    f"./results/economic_results/o_info/images/all/pdf/PDF_all_together.pdf",
    dpi=600,
    bbox_inches="tight",
)

plt.show()

<a name="o_info_full_length_pdf_friedman"></a>
##### Friedman test

In [ ]:
import scikit_posthocs as sp

In [ ]:
with open(
    "./results/economic_results/o_info/dataframes/all/dict_proba_density_function_all.pickle",
    "rb",
) as file:
    diz_pdf = pickle.load(file)

In [ ]:
def friedman_test(*groups):
    stat, p_value = friedmanchisquare(*groups)
    return stat, p_value

In [ ]:
nemenyi_results = {}
friedman_results = []
p_critic = 0.05

for sector in sectors:
    group_3 = diz_pdf[sector]["3"]
    group2 = diz_pdf[sector]["2_1"]
    group3 = diz_pdf[sector]["1_1_1"]
    
    group_2_1 = random.sample(group2, len(group_3))
    group_1_1_1 = random.sample(group3, len(group_3))

    # Perform Friedman test
    stat, p_value = friedman_test(group_3, group_2_1, group_1_1_1)
    friedman_results.append((stat, p_value))

    print(f"Sector: {sector}")
    print(f"Friedman statistic and P-value: {stat}, {p_value}")

    if p_value < p_critic:
        # Perform Nemenyi post hoc test for multiple comparisons
        results = sp.posthoc_nemenyi([group_3, group_2_1, group_1_1_1])
        print("Nemenyi test results:")
        print(results)
        nemenyi_results[sector] = results

    print("\n")

In [ ]:
nemenyi_results["Industrials"]

In [ ]:
nemenyi_results["Industrials"][1][2]

In [ ]:
plt.plot([1, 2, 3, 4], [1, 4, 9, 16])

# Aggiunta di un simbolo "*" sopra la linea orizzontale
plt.annotate('*', xy=(2, 5), xytext=(3, 15),
            arrowprops=dict(facecolor='black', shrink=0.05))

plt.show()

In [ ]:
from matplotlib.ticker import MaxNLocator

fig, axs = plt.subplots(2, 4, figsize=(15, 8))

for index, sector in enumerate(sectors):
    keys = list(diz_pdf[sector].keys())
    data = [diz_pdf[sector][key] for key in keys]
    means = [np.mean(data[i]) for i in range(len(data))]

    row = index // 4
    col = index % 4

    ax = axs[row, col]

    for i, key in enumerate(keys):
        values_bin, bin_edge = np.histogram(data[i], bins=20)

        if i == 0:
            label = str(f"3 in {sector}")
        elif i == 1:
            label = str(f"2 in {sector}")
        else:
            label = str(f"1 in {sector}")

        ax.bar(
            bin_edge[:-1],
            values_bin / sum(values_bin),
            width=np.diff(bin_edge),
            align="edge",
            alpha=0.6,
            label=label,
        )

    ax.axvline(x=means[0], color="blue", linestyle="--")
    ax.axvline(x=means[1], color="orange", linestyle="--")
    ax.axvline(x=means[2], color="green", linestyle="--")

    ax.set_title(f"PDF of {sector}", size=17)
    ax.legend(fontsize="small")

    # Aggiungi le etichette per i p-value
    ax.annotate(
        f"P-value 3 vs 2_1: {results.iloc[0, 1]:.2f}",
        xy=(0.05, 0.9),
        xycoords="axes fraction",
        fontsize=10,
        color="red" if results.iloc[0, 1] < p_critic else "black",
    )
    ax.annotate(
        f"P-value 3 vs 1_1_1: {results.iloc[0, 2]:.2f}",
        xy=(0.05, 0.8),
        xycoords="axes fraction",
        fontsize=10,
        color="red" if results.iloc[0, 2] < p_critic else "black",
    )
    ax.annotate(
        f"P-value 2_1 vs 1_1_1: {results.iloc[1, 2]:.2f}",
        xy=(0.05, 0.7),
        xycoords="axes fraction",
        fontsize=10,
        color="red" if results.iloc[1, 2] < p_critic else "black",
    )

fig.text(0.5, -0.02, "O-information", ha="center", va="center", fontsize=22)
fig.text(-0.015, 0.5, "Probability", rotation="vertical", va="center", fontsize=22)

# Assicurati che i subplot siano ben allineati
plt.tight_layout()

#plt.savefig(
#    f"./results/economic_results/o_info/images/all/pdf/PDF_all_together.pdf",
#    dpi=600,
#    bbox_inches="tight",
#)

plt.show()

<a name="o_info_full_length_null_model"></a>
#### Null model

In [ ]:
# DO NOT RUN IF YOU ALREADY HAVE THE DATA

for target in sectors:
    oinfo_one_sector = get_o_infos_same(
        target, binarized_financial_ts_gauss, total=True
    )
    np.save(
        f"./results/economic_results/o_info/metrics/3/all/gaussian/one_sector_{target}.npy",
        oinfo_one_sector,
    )

for target in sectors:
    other_sectors = [sector for sector in sectors if sector != target]
    for i, sector in enumerate(other_sectors):

        oinfo_two_sectors = get_o_infos_two_different(
            target, sector, binarized_financial_ts_gauss, total=True
        )
        np.save(
            f"./results/economic_results/o_info/metrics/2_1/all/gaussian/one_sector_{target}_two_sector_{sector}.npy",
            oinfo_two_sectors,
        )

for target in sectors:
    other_sectors = [sector for sector in sectors if sector != target]
    for sector1 in other_sectors:
        other_sectors2 = [sector for sector in other_sectors if sector != sector1]
        for sector2 in other_sectors2:

            oinfo_three_sectors = get_o_infos_three_different(
                target, sector1, sector2, binarized_financial_ts_gauss, total=True
            )
            np.save(
                f"./results/economic_results/o_info/metrics/1_1_1/all/gaussian/one_sector_{target}_one_sector_{sector1}_one_sector_{sector2}.npy",
                oinfo_three_sectors,
            )

In [ ]:
# DO NOT RUN IF YOU ALREADY HAVE THE DATAFRAMES

for target1 in sectors:
    df_matrix = pd.DataFrame(columns=sectors)
    for target2 in sectors:
        for source in sectors:

            if target2 == target1 and source == target1:
                oinfo_one_sector = np.load(
                    f"./results/economic_results/o_info/metrics/3/all/gaussian/one_sector_{target2}.npy"
                )
                df_matrix.at[target2, source] = np.mean(oinfo_one_sector)

            elif target2 == target1 and source != target1:
                oinfo_two_sectors = np.load(
                    f"./results/economic_results/o_info/metrics/2_1/all/gaussian/one_sector_{source}_two_sector_{target2}.npy"
                )
                df_matrix.at[target2, source] = np.mean(oinfo_two_sectors)

            elif target2 != target1 and source == target1:
                oinfo_two_sectors = np.load(
                    f"./results/economic_results/o_info/metrics/2_1/all/gaussian/one_sector_{target2}_two_sector_{source}.npy"
                )
                df_matrix.at[target2, source] = np.mean(oinfo_two_sectors)

            elif target2 != target1 and source == target2:
                oinfo_two_sectors = np.load(
                    f"./results/economic_results/o_info/metrics/2_1/all/gaussian/one_sector_{target1}_two_sector_{target2}.npy"
                )
                df_matrix.at[target2, source] = np.mean(oinfo_two_sectors)

            elif target2 != target1 and source != target2:
                oinfo_three_sectors = np.load(
                    f"./results/economic_results/o_info/metrics/1_1_1/all/gaussian/one_sector_{target1}_one_sector_{target2}_one_sector_{source}.npy"
                )
                df_matrix.at[target2, source] = np.mean(oinfo_three_sectors)

    df_matrix.to_csv(
        f"./results/economic_results/o_info/dataframes/all/gaussian/matrix_{target1}.csv",
        index=True,
    )

In [ ]:
for sector in sectors:
    df = pd.read_csv(
        f"./results/economic_results/o_info/dataframes/all/gaussian/matrix_{sector}.csv",
        index_col=0,
    )
    sns.heatmap(df, cmap="Blues")

    plt.xlabel("Source", fontsize=13)
    plt.ylabel("Target", fontsize=13)
    plt.title(f"1 in {sector}, 1 in Target, 1 in Source", fontsize=20)

    plt.savefig(
        f"./results/economic_results/o_info/images/all/gaussian/heatmap_matrix_{sector}.pdf",
        dpi=600,
        bbox_inches="tight",
    )

    plt.show()

In [ ]:
mini, maxi = 50, -50
for sector in sectors:
    df = pd.read_csv(
        f"./results/economic_results/o_info/dataframes/all/gaussian/matrix_{sector}.csv",
        index_col=0,
    )
    if min(df.min()) < mini:
        mini = min(df.min())
    if max(df.max()) > maxi:
        maxi = max(df.max())

fig, axs = plt.subplots(2, 4, figsize=(20, 10))

for index, sector in enumerate(sectors):
    row = index // 4
    col = index % 4

    ax = axs[row, col]

    df = pd.read_csv(
        f"./results/economic_results/o_info/dataframes/all/gaussian/matrix_{sector}.csv",
        index_col=0,
    )
    sns.heatmap(df, ax=ax, cmap="Blues", vmin=mini, vmax=maxi)

    ax.set_title(f"{sector} (null model)", size=19)

    ax.set_xticklabels([])
    ax.set_yticklabels([])

fig.text(0.485, -0.02, "Source", ha="center", va="center", fontsize=24)
fig.text(-0.02, 0.49, "Target", rotation="vertical", va="center", fontsize=24)

plt.tight_layout()

plt.savefig(
    f"./results/economic_results/o_info/images/all/gaussian/heatmap_matrix_all_sectors.pdf",
    dpi=600,
    bbox_inches="tight",
)

plt.show()

In [ ]:
mini, maxi = 50, -50
for sector in sectors:
    df_original = pd.read_csv(
        f"./results/economic_results/o_info/dataframes/all/matrix_{sector}.csv",
        index_col=0,
    )
    if min(df_original.min()) < mini:
        mini = min(df_original.min())
    if max(df_original.max()) > maxi:
        maxi = max(df_original.max())

    df_gauss = pd.read_csv(
        f"./results/economic_results/o_info/dataframes/all/gaussian/matrix_{sector}.csv",
        index_col=0,
    )
    if min(df_gauss.min()) < mini:
        mini = min(df_gauss.min())
    if max(df_gauss.max()) > maxi:
        maxi = max(df_gauss.max())

fig, axs = plt.subplots(2, 8, figsize=(40, 10))

for index, sector in enumerate(sectors):
    col = index % 8

    ax = axs[0, col]
    df_original = pd.read_csv(
        f"./results/economic_results/o_info/dataframes/all/matrix_{sector}.csv",
        index_col=0,
    )
    sns.heatmap(df_original, ax=ax, cmap="Blues", vmin=mini, vmax=maxi)
    ax.set_title(f"{sector} (original)", size=18)
    ax.set_xticklabels([])
    ax.set_yticklabels([])

    ax = axs[1, col]
    df_gauss = pd.read_csv(
        f"./results/economic_results/o_info/dataframes/all/gaussian/matrix_{sector}.csv",
        index_col=0,
    )
    sns.heatmap(df_gauss, ax=ax, cmap="Blues", vmin=mini, vmax=maxi)
    ax.set_title(f"{sector} (null model)", size=18)
    ax.set_xticklabels([])
    ax.set_yticklabels([])

fig.text(0.495, -0.02, "Source", ha="center", va="center", fontsize=30)
fig.text(-0.01, 0.485, "Target", rotation="vertical", va="center", fontsize=30)

plt.tight_layout()

plt.savefig(
    f"./results/economic_results/o_info/images/all/gaussian/heatmap_matrix_comparison_original_gaussian_same_scale.pdf",
    dpi=600,
    bbox_inches="tight",
)

plt.show()

In [ ]:
mini_gauss, maxi_gauss = 50, -50
mini_original, maxi_original = 50, -50

for sector in sectors:
    df_gauss = pd.read_csv(
        f"./results/economic_results/o_info/dataframes/all/gaussian/matrix_{sector}.csv",
        index_col=0,
    )
    if min(df_gauss.min()) < mini_gauss:
        mini_gauss = min(df_gauss.min())
    if max(df_gauss.max()) > maxi_gauss:
        maxi_gauss = max(df_gauss.max())

for sector in sectors:
    df_original = pd.read_csv(
        f"./results/economic_results/o_info/dataframes/all/matrix_{sector}.csv",
        index_col=0,
    )
    if min(df_original.min()) < mini_original:
        mini_original = min(df_original.min())
    if max(df_original.max()) > maxi_original:
        maxi_original = max(df_original.max())

fig, axs = plt.subplots(2, 8, figsize=(40, 10))

for index, sector in enumerate(sectors):
    col = index % 8

    ax = axs[0, col]
    df_original = pd.read_csv(
        f"./results/economic_results/o_info/dataframes/all/matrix_{sector}.csv",
        index_col=0,
    )
    sns.heatmap(
        df_original, ax=ax, cmap="Blues", vmin=mini_original, vmax=maxi_original
    )
    ax.set_title(f"{sector} (original)", size=18)
    ax.set_xticklabels([])
    ax.set_yticklabels([])

    ax = axs[1, col]
    df_gauss = pd.read_csv(
        f"./results/economic_results/o_info/dataframes/all/gaussian/matrix_{sector}.csv",
        index_col=0,
    )
    sns.heatmap(df_gauss, ax=ax, cmap="Blues", vmin=mini_gauss, vmax=maxi_gauss)
    ax.set_title(f"{sector} (null model)", size=18)
    ax.set_xticklabels([])
    ax.set_yticklabels([])

fig.text(0.495, -0.02, "Source", ha="center", va="center", fontsize=30)
fig.text(-0.01, 0.485, "Target", rotation="vertical", va="center", fontsize=30)

plt.tight_layout()

plt.savefig(
    f"./results/economic_results/o_info/images/all/gaussian/heatmap_matrix_comparison_original_gaussian_different_scale.pdf",
    dpi=600,
    bbox_inches="tight",
)

plt.show()

<a name="o_info_full_length_null_model_stat"></a>
##### Statistical significance

<a name="o_info_full_length_null_model_stat_t_test"></a>
###### t-test

In [ ]:
# DO NOT RUN IF YOU ALREADY HAVE THE DATAFRAMES

for target1 in sectors:
    df_matrix_t_value = pd.DataFrame(columns=sectors)
    df_matrix_p_value = pd.DataFrame(columns=sectors)

    for target2 in sectors:
        for source in sectors:

            if target2 == target1 and source == target1:
                oinfo_one_sector_original = np.load(
                    f"./results/economic_results/o_info/metrics/3/all/one_sector_{target2}.npy"
                )
                oinfo_one_sector_gauss = np.load(
                    f"./results/economic_results/o_info/metrics/3/all/gaussian/one_sector_{target2}.npy"
                )
                t, p = t_test(oinfo_one_sector_original, oinfo_one_sector_gauss)
                df_matrix_t_value.at[target2, source] = t
                df_matrix_p_value.at[target2, source] = p

            elif target2 == target1 and source != target1:
                oinfo_two_sectors_original = np.load(
                    f"./results/economic_results/o_info/metrics/2_1/all/one_sector_{source}_two_sector_{target2}.npy"
                )
                oinfo_two_sectors_gauss = np.load(
                    f"./results/economic_results/o_info/metrics/2_1/all/gaussian/one_sector_{source}_two_sector_{target2}.npy"
                )
                t, p = t_test(oinfo_two_sectors_original, oinfo_two_sectors_gauss)
                df_matrix_t_value.at[target2, source] = t
                df_matrix_p_value.at[target2, source] = p

            elif target2 != target1 and source == target1:
                oinfo_two_sectors_original = np.load(
                    f"./results/economic_results/o_info/metrics/2_1/all/one_sector_{target2}_two_sector_{source}.npy"
                )
                oinfo_two_sectors_gauss = np.load(
                    f"./results/economic_results/o_info/metrics/2_1/all/gaussian/one_sector_{target2}_two_sector_{source}.npy"
                )
                t, p = t_test(oinfo_two_sectors_original, oinfo_two_sectors_gauss)
                df_matrix_t_value.at[target2, source] = t
                df_matrix_p_value.at[target2, source] = p

            elif target2 != target1 and source == target2:
                oinfo_two_sectors_original = np.load(
                    f"./results/economic_results/o_info/metrics/2_1/all/one_sector_{target1}_two_sector_{target2}.npy"
                )
                oinfo_two_sectors_gauss = np.load(
                    f"./results/economic_results/o_info/metrics/2_1/all/gaussian/one_sector_{target1}_two_sector_{target2}.npy"
                )
                t, p = t_test(oinfo_two_sectors_original, oinfo_two_sectors_gauss)
                df_matrix_t_value.at[target2, source] = t
                df_matrix_p_value.at[target2, source] = p

            elif target2 != target1 and source != target2:
                oinfo_three_sectors_original = np.load(
                    f"./results/economic_results/o_info/metrics/1_1_1/all/one_sector_{target1}_one_sector_{target2}_one_sector_{source}.npy"
                )
                oinfo_three_sectors_gauss = np.load(
                    f"./results/economic_results/o_info/metrics/1_1_1/all/gaussian/one_sector_{target1}_one_sector_{target2}_one_sector_{source}.npy"
                )
                t, p = t_test(oinfo_three_sectors_original, oinfo_three_sectors_gauss)
                df_matrix_t_value.at[target2, source] = t
                df_matrix_p_value.at[target2, source] = p

    df_matrix_t_value.to_csv(
        f"./results/economic_results/o_info/dataframes/all/gaussian/statistical_significance/matrix_t_values_{target1}.csv",
        index=True,
    )
    df_matrix_p_value.to_csv(
        f"./results/economic_results/o_info/dataframes/all/gaussian/statistical_significance/matrix_t_p_values_{target1}.csv",
        index=True,
    )

In [ ]:
fig, axs = plt.subplots(2, 4, figsize=(20, 10))

for index, sector in enumerate(sectors):
    row = index // 4
    col = index % 4

    ax = axs[row, col]

    df_t = pd.read_csv(
        f"./results/economic_results/o_info/dataframes/all/gaussian/statistical_significance/matrix_t_values_{sector}.csv",
        index_col=0,
    )
    df_t = df_t.applymap(sign_t)
    df_p = pd.read_csv(
        f"./results/economic_results/o_info/dataframes/all/gaussian/statistical_significance/matrix_t_p_values_{sector}.csv",
        index_col=0,
    )
    df_p = df_p.applymap(sign_p)
    
    cmap = sns.color_palette(["lightblue"])
    
    force_break_loop = False
    for column in df_p.columns:
        for row in df_p.columns:
            if df_p[column][row] == "red":
                cmap = sns.color_palette(["white", "lightblue"])
                force_break_loop = True
                break
        if force_break_loop:
            break
            
    sns.heatmap(
        df_p.replace({"red": 0, "green": 1}),
        cmap=cmap,
        annot=df_t,
        fmt="s",
        cbar=False,
        ax=ax,
    )

    ax.set_title(f"{sector} (t-test)", size=19)

    ax.set_xticklabels([])
    ax.set_yticklabels([])

fig.text(0.5, -0.02, "Source", ha="center", va="center", fontsize=25)
fig.text(-0.02, 0.5, "Target", rotation="vertical", va="center", fontsize=25)

plt.tight_layout()

plt.savefig(
    f"./results/economic_results/o_info/images/all/gaussian/statistical_significance/heatmap_matrix_t_test_all_sectors.pdf",
    dpi=600,
    bbox_inches="tight",
)

plt.show()

<a name="o_info_full_length_null_model_stat_Z_score"></a>
###### Z-score

In [ ]:
# DO NOT RUN IF YOU ALREADY HAVE THE DATAFRAMES

for target1 in sectors:
    df_matrix_z_score = pd.DataFrame(columns=sectors)
    df_matrix_p_value = pd.DataFrame(columns=sectors)
    for target2 in sectors:
        for source in sectors:

            if target2 == target1 and source == target1:
                oinfo_one_sector_original = np.load(
                    f"./results/economic_results/o_info/metrics/3/all/one_sector_{target2}.npy"
                )
                oinfo_one_sector_gauss = np.load(
                    f"./results/economic_results/o_info/metrics/3/all/gaussian/one_sector_{target2}.npy"
                )
                z, p = z_score(oinfo_one_sector_original, oinfo_one_sector_gauss)
                df_matrix_z_score.at[target2, source] = z
                df_matrix_p_value.at[target2, source] = p

            elif target2 == target1 and source != target1:
                oinfo_two_sectors_original = np.load(
                    f"./results/economic_results/o_info/metrics/2_1/all/one_sector_{source}_two_sector_{target2}.npy"
                )
                oinfo_two_sectors_gauss = np.load(
                    f"./results/economic_results/o_info/metrics/2_1/all/gaussian/one_sector_{source}_two_sector_{target2}.npy"
                )
                z, p = z_score(oinfo_two_sectors_original, oinfo_two_sectors_gauss)
                df_matrix_z_score.at[target2, source] = z
                df_matrix_p_value.at[target2, source] = p

            elif target2 != target1 and source == target1:
                oinfo_two_sectors_original = np.load(
                    f"./results/economic_results/o_info/metrics/2_1/all/one_sector_{target2}_two_sector_{source}.npy"
                )
                oinfo_two_sectors_gauss = np.load(
                    f"./results/economic_results/o_info/metrics/2_1/all/gaussian/one_sector_{target2}_two_sector_{source}.npy"
                )
                z, p = z_score(oinfo_two_sectors_original, oinfo_two_sectors_gauss)
                df_matrix_z_score.at[target2, source] = z
                df_matrix_p_value.at[target2, source] = p

            elif target2 != target1 and source == target2:
                oinfo_two_sectors_original = np.load(
                    f"./results/economic_results/o_info/metrics/2_1/all/one_sector_{target1}_two_sector_{target2}.npy"
                )
                oinfo_two_sectors_gauss = np.load(
                    f"./results/economic_results/o_info/metrics/2_1/all/gaussian/one_sector_{target1}_two_sector_{target2}.npy"
                )
                z, p = z_score(oinfo_two_sectors_original, oinfo_two_sectors_gauss)
                df_matrix_z_score.at[target2, source] = z
                df_matrix_p_value.at[target2, source] = p

            elif target2 != target1 and source != target2:
                oinfo_three_sectors_original = np.load(
                    f"./results/economic_results/o_info/metrics/1_1_1/all/one_sector_{target1}_one_sector_{target2}_one_sector_{source}.npy"
                )
                oinfo_three_sectors_gauss = np.load(
                    f"./results/economic_results/o_info/metrics/1_1_1/all/gaussian/one_sector_{target1}_one_sector_{target2}_one_sector_{source}.npy"
                )
                z, p = z_score(oinfo_three_sectors_original, oinfo_three_sectors_gauss)
                df_matrix_z_score.at[target2, source] = z
                df_matrix_p_value.at[target2, source] = p

    df_matrix_z_score.to_csv(
        f"./results/economic_results/o_info/dataframes/all/gaussian/statistical_significance/matrix_Z_score_{target1}.csv",
        index=True,
    )
    df_matrix_p_value.to_csv(
        f"./results/economic_results/o_info/dataframes/all/gaussian/statistical_significance/matrix_Z_p_value_{target1}.csv",
        index=True,
    )

In [ ]:
mini, maxi = 50, -50
for sector in sectors:
    df = pd.read_csv(
        f"./results/economic_results/o_info/dataframes/all/gaussian/statistical_significance/matrix_Z_score_{sector}.csv",
        index_col=0,
    )
    if min(df.min()) < mini:
        mini = min(df.min())
    if max(df.max()) > maxi:
        maxi = max(df.max())

fig, axs = plt.subplots(2, 4, figsize=(20, 10))

for index, sector in enumerate(sectors):
    row = index // 4
    col = index % 4

    ax = axs[row, col]

    df_z = pd.read_csv(
        f"./results/economic_results/o_info/dataframes/all/gaussian/statistical_significance/matrix_Z_score_{sector}.csv",
        index_col=0,
    )

    sns.heatmap(df_z, cmap="coolwarm", ax=ax, vmin=mini, vmax=maxi)

    ax.set_title(f"{sector} (Z-score)", size=19)

    ax.set_xticklabels([])
    ax.set_yticklabels([])

fig.text(0.483, -0.02, "Source", ha="center", va="center", fontsize=25)
fig.text(-0.02, 0.49, "Target", rotation="vertical", va="center", fontsize=25)

plt.tight_layout()

plt.savefig(
    f"./results/economic_results/o_info/images/all/gaussian/statistical_significance/heatmap_matrix_Z_score_all_sectors.pdf",
    dpi=600,
    bbox_inches="tight",
)

plt.show()

<a name="o_info_full_length_null_model_stat_comparison"></a>
###### Comparison

In [ ]:
for sector in sectors:
    plt.figure(figsize=(6, 4))
    
    df_z = pd.read_csv(
        f"./results/economic_results/o_info/dataframes/all/gaussian/statistical_significance/matrix_Z_score_{sector}.csv",
        index_col=0,
    )
    df_t = pd.read_csv(
        f"./results/economic_results/o_info/dataframes/all/gaussian/statistical_significance/matrix_t_values_{sector}.csv",
        index_col=0,
    )
    
    for target in sectors:
        plt.scatter(df_z[target], df_t[target], c="blue")

    plt.title(f"Z-score - t-test for {sector}")
    plt.xlabel("Z-score")
    plt.ylabel("t-test")

    plt.savefig(
        f"./results/economic_results/o_info/images/all/gaussian/statistical_significance/scatterplot_Z_score_t_test_sector_{sector}.pdf",
        dpi=600,
        bbox_inches="tight",
    )

    plt.show()

In [ ]:
plt.figure(figsize=(6, 4))

for sector in sectors:

    df_z = pd.read_csv(
        f"./results/economic_results/o_info/dataframes/all/gaussian/statistical_significance/matrix_Z_score_{sector}.csv",
        index_col=0,
    )
    df_t = pd.read_csv(
        f"./results/economic_results/o_info/dataframes/all/gaussian/statistical_significance/matrix_t_values_{sector}.csv",
        index_col=0,
    )

    for target in sectors:
        plt.scatter(df_z[target], df_t[target], c="blue")

plt.title(f"Scatterplot Z-score - t-test")
plt.xlabel("Z-score")
plt.ylabel("t-test")

plt.savefig(
   f"./results/economic_results/o_info/images/all/gaussian/statistical_significance/scatterplot_Z_score_t_test_one_plot.pdf",
   dpi=600,
   bbox_inches="tight",
)

plt.show()

In [ ]:
fig, axs = plt.subplots(2, 4, figsize=(20, 10))

for i, sector in enumerate(sectors):
    df_z = pd.read_csv(
        f"./results/economic_results/o_info/dataframes/all/gaussian/statistical_significance/matrix_Z_score_{sector}.csv",
        index_col=0,
    )
    df_t = pd.read_csv(
        f"./results/economic_results/o_info/dataframes/all/gaussian/statistical_significance/matrix_t_values_{sector}.csv",
        index_col=0,
    )

    row = i // 4
    col = i % 4

    for target in sectors:
        axs[row, col].scatter(df_z[target], df_t[target], c="blue")
    axs[row, col].set_title(f"Z-score - t-test for {sector}", size=17)
    axs[row, col].set_xlabel("Z-score", size=14)
    axs[row, col].set_ylabel("t-test", size=14)

plt.tight_layout()
plt.savefig(
    f"./results/economic_results/o_info/images/all/gaussian/statistical_significance/scatterplot_Z_score_t_test_subplots.pdf",
    dpi=600,
    bbox_inches="tight",
)
plt.show()

In [ ]:
fig, axs = plt.subplots(2, 4, figsize=(20, 10))

for i, sector in enumerate(sectors):
    df_z = pd.read_csv(
        f"./results/economic_results/o_info/dataframes/all/gaussian/statistical_significance/matrix_Z_p_value_{sector}.csv",
        index_col=0,
    )
    df_t = pd.read_csv(
        f"./results/economic_results/o_info/dataframes/all/gaussian/statistical_significance/matrix_t_p_values_{sector}.csv",
        index_col=0,
    )

    row = i // 4
    col = i % 4

    for target in sectors:
        axs[row, col].scatter(df_z[target], df_t[target], c="blue")
    axs[row, col].axhline(y=0.05, color="r", linestyle="--")

    axs[row, col].set_title(f"p-values Z-score - t-test for {sector}", size=16)
    axs[row, col].set_xlabel("p-value Z-score", size=14)
    axs[row, col].set_ylabel("p-value t-test", size=14)

plt.tight_layout()
plt.savefig(
   f"./results/economic_results/o_info/images/all/gaussian/statistical_significance/scatterplot_p_values_Z_score_t_test_subplots.pdf",
   dpi=600,
   bbox_inches="tight",
)
plt.show()

<a name="o_info_full_length_null_model_no_multivariate"></a>
#### Null model no multivariate

In [ ]:
mean_values = financial_ts.iloc[:, 1:].mean()
variance_values = financial_ts.iloc[:, 1:].var()

generated_data_list = []

for column in financial_ts.columns[1:]:
    mean = mean_values[column]
    variance = variance_values[column]

    generated_data = np.random.normal(mean, np.sqrt(variance), financial_ts.shape[0])
    generated_data_list.append(pd.Series(generated_data, name=column))

financial_ts_gauss_no_multivariate = pd.concat(
    [financial_ts["Date"]] + generated_data_list, axis=1
)

In [ ]:
# DO NOT RUN IF YOU ALREADY HAVE THE DATA

for target in sectors:
    oinfo_one_sector = get_o_infos_same(
        target, financial_ts_gauss_no_multivariate, total=True
    )
    np.save(
        f"./results/economic_results/o_info/metrics/3/all/gaussian/no_multivariate/one_{target}.npy",
        oinfo_one_sector,
    )

for target in sectors:
    other_sectors = [sector for sector in sectors if sector != target]
    for i, sector in enumerate(other_sectors):
        oinfo_two_sectors = get_o_infos_two_different(
            target, sector, financial_ts_gauss_no_multivariate, total=True
        )
        np.save(
            f"./results/economic_results/o_info/metrics/2_1/all/gaussian/no_multivariate/one_{target}_two_{sector}.npy",
            oinfo_two_sectors,
        )

for target in sectors:
    other_sectors = [sector for sector in sectors if sector != target]
    for sector1 in other_sectors:
        other_sectors2 = [sector for sector in other_sectors if sector != sector1]
        for sector2 in other_sectors2:
            oinfo_three_sectors = get_o_infos_three_different(
                target, sector1, sector2, financial_ts_gauss_no_multivariate, total=True
            )
            np.save(
                f"./results/economic_results/o_info/metrics/1_1_1/all/gaussian/no_multivariate/one_{target}_one_{sector1}_one_{sector2}.npy",
                oinfo_three_sectors,
            )

In [ ]:
# DO NOT RUN IF YOU ALREADY HAVE THE DATAFRAMES

for target1 in sectors:
    df_matrix = pd.DataFrame(columns=sectors)
    for target2 in sectors:
        for source in sectors:

            if target2 == target1 and source == target1:
                oinfo_one_sector = np.load(
                    f"./results/economic_results/o_info/metrics/3/all/gaussian/no_multivariate/one_{target2}.npy"
                )
                df_matrix.at[target2, source] = np.mean(oinfo_one_sector)

            elif target2 == target1 and source != target1:
                oinfo_two_sectors = np.load(
                    f"./results/economic_results/o_info/metrics/2_1/all/gaussian/no_multivariate/one_{source}_two_{target2}.npy"
                )
                df_matrix.at[target2, source] = np.mean(oinfo_two_sectors)

            elif target2 != target1 and source == target1:
                oinfo_two_sectors = np.load(
                    f"./results/economic_results/o_info/metrics/2_1/all/gaussian/no_multivariate/one_{target2}_two_{source}.npy"
                )
                df_matrix.at[target2, source] = np.mean(oinfo_two_sectors)

            elif target2 != target1 and source == target2:
                oinfo_two_sectors = np.load(
                    f"./results/economic_results/o_info/metrics/2_1/all/gaussian/no_multivariate/one_{target1}_two_{target2}.npy"
                )
                df_matrix.at[target2, source] = np.mean(oinfo_two_sectors)

            elif target2 != target1 and source != target2:
                oinfo_three_sectors = np.load(
                    f"./results/economic_results/o_info/metrics/1_1_1/all/gaussian/no_multivariate/one_{target1}_one_{target2}_one_{source}.npy"
                )
                df_matrix.at[target2, source] = np.mean(oinfo_three_sectors)

    df_matrix.to_csv(
        f"./results/economic_results/o_info/dataframes/all/gaussian/no_multivariate/matrix_{target1}.csv",
        index=True,
    )

In [ ]:
sector = "Industrials"
df_matrix = pd.read_csv(
    f"./results/economic_results/o_info/dataframes/all/gaussian/no_multivariate/matrix_{sector}.csv",
    index_col=0,
)
df_matrix

In [ ]:
mini, maxi = 50, -50
for sector in sectors:
    df = pd.read_csv(
        f"./results/economic_results/o_info/dataframes/all/gaussian/no_multivariate/matrix_{sector}.csv",
        index_col=0,
    )
    if min(df.min()) < mini:
        mini = min(df.min())
    if max(df.max()) > maxi:
        maxi = max(df.max())

fig, axs = plt.subplots(2, 4, figsize=(20, 10))

for index, sector in enumerate(sectors):
    row = index // 4
    col = index % 4

    ax = axs[row, col]

    df = pd.read_csv(
        f"./results/economic_results/o_info/dataframes/all/gaussian/no_multivariate/matrix_{sector}.csv",
        index_col=0,
    )
    sns.heatmap(df, ax=ax, cmap="Blues", vmin=mini, vmax=maxi)

    ax.set_title(f"{sector} (no multivariate)", size=19)

    ax.set_xticklabels([])
    ax.set_yticklabels([])

fig.text(0.485, -0.02, "Source", ha="center", va="center", fontsize=24)
fig.text(-0.02, 0.49, "Target", rotation="vertical", va="center", fontsize=24)

plt.tight_layout()

#plt.savefig(
#    f"./results/economic_results/o_info/images/all/gaussian/no_multivariate/heatmap_matrix_all_sectors.pdf",
#    dpi=600,
#    bbox_inches="tight",
#)

plt.show()

<a name="o_info_divided_year"></a>
### Divided by year

In [ ]:
financial_ts["Date"] = pd.to_datetime(financial_ts["Date"])
years = financial_ts["Date"].dt.year.unique()[1:]

diz_one_year = {}

for year in years:
    df_year = financial_ts[financial_ts["Date"].dt.year == year]
    diz_one_year[year] = df_year

In [ ]:
tw = 7
diz_one_year_bin = {}

for year in years:
    diz_one_year_bin[year] = binarize_df(diz_one_year[year], tw)

In [ ]:
# DO NOT RUN IF YOU ALREADY HAVE THE DATA

for year in years:
    for target in sectors:
        oinfo_one_sector = get_o_infos_same(target, diz_one_year_bin[year], total=True)
        np.save(
            f"./results/economic_results/o_info/metrics/3/divided_years/one_sector_{target}_year_{year}.npy",
            oinfo_one_sector,
        )

for year in years:
    for target in sectors:
        other_sectors = [sector for sector in sectors if sector != target]
        for i, sector in enumerate(other_sectors):

            oinfo_two_sectors = get_o_infos_two_different(
                target, sector, diz_one_year_bin[year], total=True
            )
            np.save(
                f"./results/economic_results/o_info/metrics/2_1/divided_years/one_sector_{target}_two_sector_{sector}_year_{year}.npy",
                oinfo_two_sectors,
            )

for year in years:
    for target in sectors:
        other_sectors = [sector for sector in sectors if sector != target]
        for sector1 in other_sectors:
            other_sectors2 = [sector for sector in other_sectors if sector != sector1]
            for sector2 in other_sectors2:

                oinfo_three_sectors = get_o_infos_three_different(
                    target, sector1, sector2, diz_one_year_bin[year], total=True
                )
                np.save(
                    f"./results/economic_results/o_info/metrics/1_1_1/divided_years/one_sector_{target}_one_sector_{sector1}_one_sector_{sector2}_year_{year}.npy",
                    oinfo_three_sectors,
                )

In [ ]:
# DO NOT RUN IF YOU ALREADY HAVE THE DATAFRAMES

for target1 in sectors:
    for target2 in sectors:
        df_one_year = pd.DataFrame(columns=years)
        for year in years:
            for source in sectors:

                if target2 == target1 and source == target1:
                    oinfo_one_sector = np.load(
                        f"./results/economic_results/o_info/metrics/3/divided_years/one_sector_{target2}_year_{year}.npy"
                    )
                    df_one_year.at[source, year] = np.mean(oinfo_one_sector)

                elif target2 == target1 and source != target1:
                    oinfo_two_sectors = np.load(
                        f"./results/economic_results/o_info/metrics/2_1/divided_years/one_sector_{source}_two_sector_{target2}_year_{year}.npy"
                    )
                    df_one_year.at[source, year] = np.mean(oinfo_two_sectors)

                elif target2 != target1 and source == target1:
                    oinfo_two_sectors = np.load(
                        f"./results/economic_results/o_info/metrics/2_1/divided_years/one_sector_{target2}_two_sector_{source}_year_{year}.npy"
                    )
                    df_one_year.at[source, year] = np.mean(oinfo_two_sectors)

                elif target2 != target1 and source == target2:
                    oinfo_two_sectors = np.load(
                        f"./results/economic_results/o_info/metrics/2_1/divided_years/one_sector_{target1}_two_sector_{target2}_year_{year}.npy"
                    )
                    df_one_year.at[source, year] = np.mean(oinfo_two_sectors)

                elif target2 != target1 and source != target2:
                    oinfo_three_sectors = np.load(
                        f"./results/economic_results/o_info/metrics/1_1_1/divided_years/one_sector_{target1}_one_sector_{target2}_one_sector_{source}_year_{year}.npy"
                    )
                    df_one_year.at[source, year] = np.mean(oinfo_three_sectors)

        df_one_year.to_csv(
            f"./results/economic_results/o_info/dataframes/divided_years/dataframe_{target1}_{target2}.csv",
            index=True,
        )

In [ ]:
for target1 in sectors:
    for target2 in sectors:

        df = pd.read_csv(
            f"./results/economic_results/o_info/dataframes/divided_years/dataframe_{target1}_{target2}.csv",
            index_col=0,
        )

        sns.heatmap(df, cmap="Blues")

        plt.axvline(x=1.5, color="brown", linestyle="--", label="Twin towers")
        plt.axvline(x=8.5, color="blue", linestyle="--", label="Financial crisis")
        plt.axvline(x=9.5, color="green", linestyle="--", label="Obama president")
        plt.axvline(
            x=11.5,
            color="yellow",
            linestyle="--",
            label="European sovereign debt crisis / Quantitative easing USA",
        )
        plt.axvline(x=17.5, color="darkorchid", linestyle="--", label="Trump president")
        plt.axvline(x=19.5, color="magenta", linestyle="--", label="Covid-19")
        plt.axvline(x=21.5, color="teal", linestyle="--", label="Biden president")

        plt.xlabel("Year", fontsize=15)
        plt.ylabel("Source", fontsize=15)
        plt.title(f"1 {target1}, 1 {target2}", fontsize=18)

        # plt.legend()

        plt.savefig(
            f"./results/economic_results/o_info/images/divided_years/heatmap_{target1}_{target2}.pdf",
            dpi=600,
            bbox_inches="tight",
        )

        plt.show()

In [ ]:
mini = [50 for i in range(len(sectors))]
maxi = [-50 for i in range(len(sectors))]

for i, target1 in enumerate(sectors):
    for target2 in sectors:
        df = pd.read_csv(
            f"./results/economic_results/o_info/dataframes/divided_years/dataframe_{target1}_{target2}.csv",
            index_col=0,
        )
        if min(df.min()) < mini[i]:
            mini[i] = min(df.min())
        if max(df.max()) > maxi[i]:
            maxi[i] = max(df.max())

for index1, sector1 in enumerate(sectors):
    fig, axs = plt.subplots(2, 4, figsize=(21, 12))

    for index2, sector2 in enumerate(sectors):
        df = pd.read_csv(
            f"./results/economic_results/o_info/dataframes/divided_years/dataframe_{sector1}_{sector2}.csv",
            index_col=0,
        )

        row = index2 // 4
        col = index2 % 4

        ax = axs[row, col]

        sns.heatmap(df, ax=ax, cmap="Blues", vmin=mini[index1], vmax=maxi[index1])

        ax.axvline(x=1.5, color="brown", linestyle="--", label="Twin towers")
        ax.axvline(x=8.5, color="blue", linestyle="--", label="Financial crisis")
        ax.axvline(x=9.5, color="green", linestyle="--", label="Obama president")
        ax.axvline(
            x=11.5,
            color="yellow",
            linestyle="--",
            label="European sovereign debt crisis / Quantitative easing USA",
        )
        ax.axvline(x=17.5, color="darkorchid", linestyle="--", label="Trump president")
        ax.axvline(x=19.5, color="magenta", linestyle="--", label="Covid-19")
        ax.axvline(x=21.5, color="teal", linestyle="--", label="Biden president")

        ax.set_title(f"{sector2}", size=25)
        ax.set_xticks(
            [0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20],
            labels=[
                "2000",
                "2002",
                "2004",
                "2006",
                "2008",
                "2010",
                "2012",
                "2014",
                "2016",
                "2018",
                "2020",
            ],
        )
        ax.set_yticklabels([])

    fig.text(0.485, -0.03, "Year", ha="center", va="center", fontsize=28)
    fig.text(-0.02, 0.48, "Sector", rotation="vertical", va="center", fontsize=28)

    plt.suptitle(f"{sector1}", size=40, x=0.485)
    plt.tight_layout()

    plt.savefig(
        f"./results/economic_results/o_info/images/divided_years/heatmap_grouped_{sector1}.pdf",
        dpi=600,
        bbox_inches="tight",
    )

    plt.show()

<a name="o_info_one_year_null_model"></a>
#### Null model

<a name="o_info_one_year_null_model_randomised_divided"></a>
##### Randomised and divided by year

In [ ]:
financial_ts_gauss["Date"] = pd.to_datetime(financial_ts_gauss["Date"])
years = financial_ts_gauss["Date"].dt.year.unique()[1:]

diz_one_year = {}

for year in years:
    df_year = financial_ts_gauss[financial_ts_gauss["Date"].dt.year == year]
    diz_one_year[year] = df_year

In [ ]:
tw = 7
diz_one_year_gaussian_bin = {}

for year in years:
    diz_one_year_gaussian_bin[year] = binarize_df(diz_one_year[year], tw)

In [ ]:
# DO NOT RUN IF YOU ALREADY HAVE THE DATA

for year in years:
    for target in sectors:
        oinfo_one_sector = get_o_infos_same(
            target, diz_one_year_gaussian_bin[year], total=True
        )
        np.save(
            f"./results/economic_results/o_info/metrics/3/divided_years/gaussian_before/one_sector_{target}_year_{year}.npy",
            oinfo_one_sector,
        )

for year in years:
    for target in sectors:
        other_sectors = [sector for sector in sectors if sector != target]
        for i, sector in enumerate(other_sectors):

            oinfo_two_sectors = get_o_infos_two_different(
                target, sector, diz_one_year_gaussian_bin[year], total=True
            )
            np.save(
                f"./results/economic_results/o_info/metrics/2_1/divided_years/gaussian_before/one_sector_{target}_two_sector_{sector}_year_{year}.npy",
                oinfo_two_sectors,
            )

for year in years:
    for target in sectors:
        other_sectors1 = [sector for sector in sectors if sector != target]
        for sector1 in other_sectors1:
            other_sectors2 = [sector for sector in other_sectors1 if sector != sector1]
            for sector2 in other_sectors2:

                oinfo_three_sectors = get_o_infos_three_different(
                    target,
                    sector1,
                    sector2,
                    diz_one_year_gaussian_bin[year],
                    total=True,
                )
                np.save(
                    f"./results/economic_results/o_info/metrics/1_1_1/divided_years/gaussian_before/one_{target}_one_{sector1}_one_{sector2}_year_{year}.npy",
                    oinfo_three_sectors,
                )

In [ ]:
# DO NOT RUN IF YOU ALREADY HAVE THE DATAFRAMES

for target1 in sectors:
    for target2 in sectors:
        df_one_year = pd.DataFrame(columns=years)
        for year in years:
            for source in sectors:

                if target2 == target1 and source == target1:
                    oinfo_one_sector = np.load(
                        f"./results/economic_results/o_info/metrics/3/divided_years/gaussian_before/one_sector_{target2}_year_{year}.npy"
                    )
                    df_one_year.at[source, year] = np.mean(oinfo_one_sector)

                elif target2 == target1 and source != target1:
                    oinfo_two_sectors = np.load(
                        f"./results/economic_results/o_info/metrics/2_1/divided_years/gaussian_before/one_sector_{source}_two_sector_{target2}_year_{year}.npy"
                    )
                    df_one_year.at[source, year] = np.mean(oinfo_two_sectors)

                elif target2 != target1 and source == target1:
                    oinfo_two_sectors = np.load(
                        f"./results/economic_results/o_info/metrics/2_1/divided_years/gaussian_before/one_sector_{target2}_two_sector_{source}_year_{year}.npy"
                    )
                    df_one_year.at[source, year] = np.mean(oinfo_two_sectors)

                elif target2 != target1 and source == target2:
                    oinfo_two_sectors = np.load(
                        f"./results/economic_results/o_info/metrics/2_1/divided_years/gaussian_before/one_sector_{target1}_two_sector_{target2}_year_{year}.npy"
                    )
                    df_one_year.at[source, year] = np.mean(oinfo_two_sectors)

                elif target2 != target1 and source != target2:
                    oinfo_three_sectors = np.load(
                        f"./results/economic_results/o_info/metrics/1_1_1/divided_years/gaussian_before/one_{target1}_one_{target2}_one_{source}_year_{year}.npy"
                    )
                    df_one_year.at[source, year] = np.mean(oinfo_three_sectors)

        df_one_year.to_csv(
            f"./results/economic_results/o_info/dataframes/divided_years/gaussian_before/dataframe_{target1}_{target2}.csv",
            index=True,
        )

In [ ]:
for target1 in sectors:
    for target2 in sectors:

        df = pd.read_csv(
            f"./results/economic_results/o_info/dataframes/divided_years/gaussian_before/dataframe_{target1}_{target2}.csv",
            index_col=0,
        )

        sns.heatmap(df, cmap="Blues")

        plt.axvline(x=1.5, color="brown", linestyle="--", label="Twin towers")
        plt.axvline(x=8.5, color="blue", linestyle="--", label="Financial crisis")
        plt.axvline(x=9.5, color="green", linestyle="--", label="Obama president")
        plt.axvline(
            x=11.5,
            color="yellow",
            linestyle="--",
            label="European sovereign debt crisis / Quantitative easing USA",
        )
        plt.axvline(x=17.5, color="darkorchid", linestyle="--", label="Trump president")
        plt.axvline(x=19.5, color="magenta", linestyle="--", label="Covid-19")
        plt.axvline(x=21.5, color="teal", linestyle="--", label="Biden president")

        plt.xlabel("Year", fontsize=15)
        plt.ylabel("Source", fontsize=15)
        plt.title(f"Before split: 1 {target1}, 1 {target2}", fontsize=18)

        # plt.legend()

        plt.savefig(
            f"./results/economic_results/o_info/images/divided_years/gaussian_before/heatmap_{target1}_{target2}.pdf",
            dpi=600,
            bbox_inches="tight",
        )

        plt.show()

In [ ]:
mini = [50 for i in range(len(sectors))]
maxi = [-50 for i in range(len(sectors))]

for i, target1 in enumerate(sectors):
    for target2 in sectors:
        df = pd.read_csv(
            f"./results/economic_results/o_info/dataframes/divided_years/gaussian_before/dataframe_{target1}_{target2}.csv",
            index_col=0,
        )
        if min(df.min()) < mini[i]:
            mini[i] = min(df.min())
        if max(df.max()) > maxi[i]:
            maxi[i] = max(df.max())

for index1, sector1 in enumerate(sectors):
    fig, axs = plt.subplots(2, 4, figsize=(21, 12))

    for index2, sector2 in enumerate(sectors):
        df = pd.read_csv(
            f"./results/economic_results/o_info/dataframes/divided_years/gaussian_before/dataframe_{sector1}_{sector2}.csv",
            index_col=0,
        )

        row = index2 // 4
        col = index2 % 4

        ax = axs[row, col]

        sns.heatmap(df, ax=ax, cmap="Blues", vmin=mini[index1], vmax=maxi[index1])

        ax.axvline(x=1.5, color="brown", linestyle="--", label="Twin towers")
        ax.axvline(x=8.5, color="blue", linestyle="--", label="Financial crisis")
        ax.axvline(x=9.5, color="green", linestyle="--", label="Obama president")
        ax.axvline(
            x=11.5,
            color="yellow",
            linestyle="--",
            label="European sovereign debt crisis / Quantitative easing USA",
        )
        ax.axvline(x=17.5, color="darkorchid", linestyle="--", label="Trump president")
        ax.axvline(x=19.5, color="magenta", linestyle="--", label="Covid-19")
        ax.axvline(x=21.5, color="teal", linestyle="--", label="Biden president")

        ax.set_title(f"{sector2}", size=25)
        ax.set_xticks(
            [0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20],
            labels=[
                "2000",
                "2002",
                "2004",
                "2006",
                "2008",
                "2010",
                "2012",
                "2014",
                "2016",
                "2018",
                "2020",
            ],
        )
        ax.set_yticklabels([])

    fig.text(0.485, -0.03, "Year", ha="center", va="center", fontsize=28)
    fig.text(-0.02, 0.48, "Sector", rotation="vertical", va="center", fontsize=28)

    plt.suptitle(f"Before split: {sector1}", size=40, x=0.485)
    plt.tight_layout()

    plt.savefig(
        f"./results/economic_results/o_info/images/divided_years/gaussian_before/heatmap_grouped_{sector1}.pdf",
        dpi=600,
        bbox_inches="tight",
    )

    plt.show()

In [ ]:
mini_gauss = [50 for i in range(len(sectors))]
maxi_gauss = [-50 for i in range(len(sectors))]

for i, target1 in enumerate(sectors):
    for target2 in sectors:
        df = pd.read_csv(
            f"./results/economic_results/o_info/dataframes/divided_years/gaussian_before/dataframe_{target1}_{target2}.csv",
            index_col=0,
        )
        if min(df.min()) < mini_gauss[i]:
            mini_gauss[i] = min(df.min())
        if max(df.max()) > maxi_gauss[i]:
            maxi_gauss[i] = max(df.max())

mini_original = [50 for i in range(len(sectors))]
maxi_original = [-50 for i in range(len(sectors))]

for i, target1 in enumerate(sectors):
    for target2 in sectors:
        df = pd.read_csv(
            f"./results/economic_results/o_info/dataframes/divided_years/dataframe_{target1}_{target2}.csv",
            index_col=0,
        )
        if min(df.min()) < mini_original[i]:
            mini_original[i] = min(df.min())
        if max(df.max()) > maxi_original[i]:
            maxi_original[i] = max(df.max())


for index1, sector1 in enumerate(sectors):
    fig, axs = plt.subplots(2, 8, figsize=(40, 12))

    for index2, sector2 in enumerate(sectors):
        col = index2 % 8

        ax = axs[0, col]
        df = pd.read_csv(
            f"./results/economic_results/o_info/dataframes/divided_years/dataframe_{sector1}_{sector2}.csv",
            index_col=0,
        )
        sns.heatmap(
            df,
            ax=ax,
            cmap="Blues",
            vmin=mini_original[index1],
            vmax=maxi_original[index1],
        )
        ax.axvline(x=1.5, color="brown", linestyle="--", label="Twin towers")
        ax.axvline(x=8.5, color="blue", linestyle="--", label="Financial crisis")
        ax.axvline(x=9.5, color="green", linestyle="--", label="Obama president")
        ax.axvline(
            x=11.5,
            color="yellow",
            linestyle="--",
            label="European sovereign debt crisis / Quantitative easing USA",
        )
        ax.axvline(x=17.5, color="darkorchid", linestyle="--", label="Trump president")
        ax.axvline(x=19.5, color="magenta", linestyle="--", label="Covid-19")
        ax.axvline(x=21.5, color="teal", linestyle="--", label="Biden president")

        ax.set_title(f"{sector2} (original)", size=22)
        ax.set_xticks(
            [0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20],
            labels=[
                "2000",
                "2002",
                "2004",
                "2006",
                "2008",
                "2010",
                "2012",
                "2014",
                "2016",
                "2018",
                "2020",
            ],
        )
        ax.set_yticklabels([])

        ax = axs[1, col]
        df = pd.read_csv(
            f"./results/economic_results/o_info/dataframes/divided_years/gaussian_before/dataframe_{sector1}_{sector2}.csv",
            index_col=0,
        )
        sns.heatmap(
            df, ax=ax, cmap="Blues", vmin=mini_gauss[index1], vmax=maxi_gauss[index1]
        )

        ax.axvline(x=1.5, color="brown", linestyle="--", label="Twin towers")
        ax.axvline(x=8.5, color="blue", linestyle="--", label="Financial crisis")
        ax.axvline(x=9.5, color="green", linestyle="--", label="Obama president")
        ax.axvline(
            x=11.5,
            color="yellow",
            linestyle="--",
            label="European sovereign debt crisis / Quantitative easing USA",
        )
        ax.axvline(x=17.5, color="darkorchid", linestyle="--", label="Trump president")
        ax.axvline(x=19.5, color="magenta", linestyle="--", label="Covid-19")
        ax.axvline(x=21.5, color="teal", linestyle="--", label="Biden president")

        ax.set_title(f"{sector2} (null)", size=22)
        ax.set_xticks(
            [0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20],
            labels=[
                "2000",
                "2002",
                "2004",
                "2006",
                "2008",
                "2010",
                "2012",
                "2014",
                "2016",
                "2018",
                "2020",
            ],
        )
        ax.set_yticklabels([])

    fig.text(0.49, -0.03, "Year", ha="center", va="center", fontsize=30)
    fig.text(-0.015, 0.465, "Sector", rotation="vertical", va="center", fontsize=30)

    plt.suptitle(f"Comparison before split: {sector1}", size=40, x=0.49)
    plt.tight_layout()

    plt.savefig(
        f"./results/economic_results/o_info/images/divided_years/gaussian_before/heatmap_grouped_comparison_original_gaussian_before_{sector1}.pdf",
        dpi=600,
        bbox_inches="tight",
    )

    plt.show()

<a name="o_info_one_year_null_model_randomised_divided_stat"></a>
###### Statistical significance

<a name="o_info_one_year_null_model_randomised_divided_stat_t_test"></a>
###### t-test

In [ ]:
financial_ts["Date"] = pd.to_datetime(financial_ts["Date"])
years = financial_ts["Date"].dt.year.unique()[1:]

In [ ]:
# DO NOT RUN IF YOU ALREADY HAVE THE DATAFRAMES

for target1 in sectors:
    for target2 in sectors:
        df_one_year_t_value = pd.DataFrame(columns=years)
        df_one_year_p_value = pd.DataFrame(columns=years)
        for year in years:
            for source in sectors:

                if target2 == target1 and source == target1:
                    oinfo_one_sector_original = np.load(
                        f"./results/economic_results/o_info/metrics/3/divided_years/one_sector_{target2}_year_{year}.npy"
                    )
                    oinfo_one_sector_gauss = np.load(
                        f"./results/economic_results/o_info/metrics/3/divided_years/gaussian_before/one_sector_{target2}_year_{year}.npy"
                    )
                    t, p = t_test(oinfo_one_sector_original, oinfo_one_sector_gauss)
                    df_one_year_t_value.at[source, year] = t
                    df_one_year_p_value.at[source, year] = p

                elif target2 == target1 and source != target1:
                    oinfo_two_sectors_original = np.load(
                        f"./results/economic_results/o_info/metrics/2_1/divided_years/one_sector_{source}_two_sector_{target2}_year_{year}.npy"
                    )
                    oinfo_two_sectors_gauss = np.load(
                        f"./results/economic_results/o_info/metrics/2_1/divided_years/gaussian_before/one_sector_{source}_two_sector_{target2}_year_{year}.npy"
                    )
                    t, p = t_test(oinfo_two_sectors_original, oinfo_two_sectors_gauss)
                    df_one_year_t_value.at[source, year] = t
                    df_one_year_p_value.at[source, year] = p

                elif target2 != target1 and source == target1:
                    oinfo_two_sectors_original = np.load(
                        f"./results/economic_results/o_info/metrics/2_1/divided_years/one_sector_{target2}_two_sector_{source}_year_{year}.npy"
                    )
                    oinfo_two_sectors_gauss = np.load(
                        f"./results/economic_results/o_info/metrics/2_1/divided_years/gaussian_before/one_sector_{target2}_two_sector_{source}_year_{year}.npy"
                    )
                    t, p = t_test(oinfo_two_sectors_original, oinfo_two_sectors_gauss)
                    df_one_year_t_value.at[source, year] = t
                    df_one_year_p_value.at[source, year] = p

                elif target2 != target1 and source == target2:
                    oinfo_two_sectors_original = np.load(
                        f"./results/economic_results/o_info/metrics/2_1/divided_years/one_sector_{target1}_two_sector_{target2}_year_{year}.npy"
                    )
                    oinfo_two_sectors_gauss = np.load(
                        f"./results/economic_results/o_info/metrics/2_1/divided_years/gaussian_before/one_sector_{target1}_two_sector_{target2}_year_{year}.npy"
                    )
                    t, p = t_test(oinfo_two_sectors_original, oinfo_two_sectors_gauss)
                    df_one_year_t_value.at[source, year] = t
                    df_one_year_p_value.at[source, year] = p

                elif target2 != target1 and source != target2:
                    oinfo_three_sectors_original = np.load(
                        f"./results/economic_results/o_info/metrics/1_1_1/divided_years/one_sector_{target1}_one_sector_{target2}_one_sector_{source}_year_{year}.npy"
                    )
                    oinfo_three_sectors_gauss = np.load(
                        f"./results/economic_results/o_info/metrics/1_1_1/divided_years/gaussian_before/one_{target1}_one_{target2}_one_{source}_year_{year}.npy"
                    )
                    t, p = t_test(
                        oinfo_three_sectors_original, oinfo_three_sectors_gauss
                    )
                    df_one_year_t_value.at[source, year] = t
                    df_one_year_p_value.at[source, year] = p

        df_one_year_t_value.to_csv(
            f"./results/economic_results/o_info/dataframes/divided_years/gaussian_before/statistical_significance/dataframe_t_values_{target1}_{target2}.csv",
            index=True,
        )
        df_one_year_p_value.to_csv(
            f"./results/economic_results/o_info/dataframes/divided_years/gaussian_before/statistical_significance/dataframe_t_p_values_{target1}_{target2}.csv",
            index=True,
        )

In [ ]:
for index1, sector1 in enumerate(sectors):
    fig, axs = plt.subplots(2, 4, figsize=(21, 12))

    for index2, sector2 in enumerate(sectors):
        row = index2 // 4
        col = index2 % 4
        ax = axs[row, col]
        
        df_t = pd.read_csv(
            f"./results/economic_results/o_info/dataframes/divided_years/gaussian_before/statistical_significance/dataframe_t_values_{sector1}_{sector2}.csv",
            index_col=0,
        )
        df_t = df_t.applymap(sign_t)
        df_p = pd.read_csv(
            f"./results/economic_results/o_info/dataframes/divided_years/gaussian_before/statistical_significance/dataframe_t_p_values_{sector1}_{sector2}.csv",
            index_col=0,
        )
        df_p = df_p.applymap(sign_p)
    
        cmap = sns.color_palette(["lightblue"])

        force_break_loop = False
        for column in df_p.columns:
            for index, row in df_p.iterrows():
                if row[column] == "red":
                    cmap = sns.color_palette(["white", "lightblue"])
                    force_break_loop = True
                    break
            if force_break_loop:
                break
                
        sns.heatmap(
            df_p.replace({"red": 0, "green": 1}),
            cmap=cmap,
            annot=df_t,
            fmt="s",
            cbar=False,
            ax=ax,
        )

        ax.axvline(x=1.5, color="brown", linestyle="--", label="Twin towers")
        ax.axvline(x=8.5, color="blue", linestyle="--", label="Financial crisis")
        ax.axvline(x=9.5, color="green", linestyle="--", label="Obama president")
        ax.axvline(
            x=11.5,
            color="yellow",
            linestyle="--",
            label="European sovereign debt crisis / Quantitative easing USA",
        )
        ax.axvline(x=17.5, color="darkorchid", linestyle="--", label="Trump president")
        ax.axvline(x=19.5, color="magenta", linestyle="--", label="Covid-19")
        ax.axvline(x=21.5, color="teal", linestyle="--", label="Biden president")

        ax.set_title(f"{sector2} (t-test)", size=22)
        ax.set_xticks(
            [0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20],
            labels=[
                "2000",
                "2002",
                "2004",
                "2006",
                "2008",
                "2010",
                "2012",
                "2014",
                "2016",
                "2018",
                "2020",
            ],
        )
        ax.set_yticklabels([])

    fig.text(0.50, -0.03, "Year", ha="center", va="center", fontsize=25)
    fig.text(-0.02, 0.472, "Sector", rotation="vertical", va="center", fontsize=25)

    plt.suptitle(f"Before split: {sector1}", size=40, x=0.50)
    plt.tight_layout()

    plt.savefig(
        f"./results/economic_results/o_info/images/divided_years/gaussian_before/statistical_significance/heatmap_grouped_before_stat_sign_t_test_{sector1}.pdf",
        dpi=600,
        bbox_inches="tight",
    )

    plt.show()

<a name="o_info_one_year_null_model_randomised_divided_stat_Z_score"></a>
###### Z-score

In [ ]:
financial_ts["Date"] = pd.to_datetime(financial_ts["Date"])
years = financial_ts["Date"].dt.year.unique()[1:]

In [ ]:
# DO NOT RUN IF YOU ALREADY HAVE THE DATAFRAMES

for target1 in sectors:
    for target2 in sectors:
        df_one_year_z_score = pd.DataFrame(columns=years)
        df_one_year_p_value = pd.DataFrame(columns=years)
        for year in years:
            for source in sectors:

                if target2 == target1 and source == target1:
                    oinfo_one_sector_original = np.load(
                        f"./results/economic_results/o_info/metrics/3/divided_years/one_sector_{target2}_year_{year}.npy"
                    )
                    oinfo_one_sector_gauss = np.load(
                        f"./results/economic_results/o_info/metrics/3/divided_years/gaussian_before/one_sector_{target2}_year_{year}.npy"
                    )
                    z, p = z_score(oinfo_one_sector_original, oinfo_one_sector_gauss)
                    df_one_year_z_score.at[source, year] = z
                    df_one_year_p_value.at[source, year] = p

                elif target2 == target1 and source != target1:
                    oinfo_two_sectors_original = np.load(
                        f"./results/economic_results/o_info/metrics/2_1/divided_years/one_sector_{source}_two_sector_{target2}_year_{year}.npy"
                    )
                    oinfo_two_sectors_gauss = np.load(
                        f"./results/economic_results/o_info/metrics/2_1/divided_years/gaussian_before/one_sector_{source}_two_sector_{target2}_year_{year}.npy"
                    )
                    z, p = z_score(oinfo_two_sectors_original, oinfo_two_sectors_gauss)
                    df_one_year_z_score.at[source, year] = z
                    df_one_year_p_value.at[source, year] = p

                elif target2 != target1 and source == target1:
                    oinfo_two_sectors_original = np.load(
                        f"./results/economic_results/o_info/metrics/2_1/divided_years/one_sector_{target2}_two_sector_{source}_year_{year}.npy"
                    )
                    oinfo_two_sectors_gauss = np.load(
                        f"./results/economic_results/o_info/metrics/2_1/divided_years/gaussian_before/one_sector_{target2}_two_sector_{source}_year_{year}.npy"
                    )
                    z, p = z_score(oinfo_two_sectors_original, oinfo_two_sectors_gauss)
                    df_one_year_z_score.at[source, year] = z
                    df_one_year_p_value.at[source, year] = p

                elif target2 != target1 and source == target2:
                    oinfo_two_sectors_original = np.load(
                        f"./results/economic_results/o_info/metrics/2_1/divided_years/one_sector_{target1}_two_sector_{target2}_year_{year}.npy"
                    )
                    oinfo_two_sectors_gauss = np.load(
                        f"./results/economic_results/o_info/metrics/2_1/divided_years/gaussian_before/one_sector_{target1}_two_sector_{target2}_year_{year}.npy"
                    )
                    z, p = z_score(oinfo_two_sectors_original, oinfo_two_sectors_gauss)
                    df_one_year_z_score.at[source, year] = z
                    df_one_year_p_value.at[source, year] = p

                elif target2 != target1 and source != target2:
                    oinfo_three_sectors_original = np.load(
                        f"./results/economic_results/o_info/metrics/1_1_1/divided_years/one_sector_{target1}_one_sector_{target2}_one_sector_{source}_year_{year}.npy"
                    )
                    oinfo_three_sectors_gauss = np.load(
                        f"./results/economic_results/o_info/metrics/1_1_1/divided_years/gaussian_before/one_{target1}_one_{target2}_one_{source}_year_{year}.npy"
                    )
                    z, p = z_score(
                        oinfo_three_sectors_original, oinfo_three_sectors_gauss
                    )
                    df_one_year_z_score.at[source, year] = z
                    df_one_year_p_value.at[source, year] = p

        df_one_year_z_score.to_csv(
            f"./results/economic_results/o_info/dataframes/divided_years/gaussian_before/statistical_significance/dataframe_z_score_{target1}_{target2}.csv",
            index=True,
        )
        df_one_year_p_value.to_csv(
            f"./results/economic_results/o_info/dataframes/divided_years/gaussian_before/statistical_significance/dataframe_z_p_value_{target1}_{target2}.csv",
            index=True,
        )

In [ ]:
mini = [50 for i in range(len(sectors))]
maxi = [-50 for i in range(len(sectors))]

for i, target1 in enumerate(sectors):
    for target2 in sectors:
        df = pd.read_csv(
            f"./results/economic_results/o_info/dataframes/divided_years/gaussian_before/statistical_significance/dataframe_z_score_{target1}_{target2}.csv",
            index_col=0,
        )
        if min(df.min()) < mini[i]:
            mini[i] = min(df.min())
        if max(df.max()) > maxi[i]:
            maxi[i] = max(df.max())

for index1, sector1 in enumerate(sectors):
    fig, axs = plt.subplots(2, 4, figsize=(21, 12))

    for index2, sector2 in enumerate(sectors):
        df_z = pd.read_csv(
            f"./results/economic_results/o_info/dataframes/divided_years/gaussian_before/statistical_significance/dataframe_z_score_{sector1}_{sector2}.csv",
            index_col=0,
        )

        row = index2 // 4
        col = index2 % 4

        ax = axs[row, col]

        sns.heatmap(
            df_z, ax=ax, cmap="coolwarm", vmin=mini[index1], vmax=maxi[index1]
        )

        ax.axvline(x=1.5, color="brown", linestyle="--", label="Twin towers")
        ax.axvline(x=8.5, color="blue", linestyle="--", label="Financial crisis")
        ax.axvline(x=9.5, color="green", linestyle="--", label="Obama president")
        ax.axvline(
            x=11.5,
            color="yellow",
            linestyle="--",
            label="European sovereign debt crisis / Quantitative easing USA",
        )
        ax.axvline(x=17.5, color="darkorchid", linestyle="--", label="Trump president")
        ax.axvline(x=19.5, color="magenta", linestyle="--", label="Covid-19")
        ax.axvline(x=21.5, color="teal", linestyle="--", label="Biden president")

        ax.set_title(f"{sector2} (Z-score)", size=22)
        ax.set_xticks(
            [0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20],
            labels=[
                "2000",
                "2002",
                "2004",
                "2006",
                "2008",
                "2010",
                "2012",
                "2014",
                "2016",
                "2018",
                "2020",
            ],
        )
        ax.set_yticklabels([])

    fig.text(0.485, -0.03, "Year", ha="center", va="center", fontsize=25)
    fig.text(-0.02, 0.478, "Sector", rotation="vertical", va="center", fontsize=25)

    plt.suptitle(f"Before split: {sector1}", size=40, x=0.485)
    plt.tight_layout()

    plt.savefig(
        f"./results/economic_results/o_info/images/divided_years/gaussian_before/statistical_significance/heatmap_grouped_before_stat_sign_Z_score_{sector1}.pdf",
        dpi=600,
        bbox_inches="tight",
    )

    plt.show()

<a name="o_info_one_year_null_model_randomised_divided_stat_comparison"></a>
###### Comparison

In [ ]:
financial_ts["Date"] = pd.to_datetime(financial_ts["Date"])
years = financial_ts["Date"].dt.year.unique()[1:]

In [ ]:
for sector1 in sectors:
    for sector2 in sectors:
        plt.figure(figsize=(6, 4))

        df_z = pd.read_csv(
            f"./results/economic_results/o_info/dataframes/divided_years/gaussian_before/statistical_significance/dataframe_z_score_{sector1}_{sector2}.csv",
            index_col=0,
        )
        df_t = pd.read_csv(
            f"./results/economic_results/o_info/dataframes/divided_years/gaussian_before/statistical_significance/dataframe_t_values_{sector1}_{sector2}.csv",
            index_col=0,
        )

        for year in years:
            plt.scatter(df_z[str(year)], df_t[str(year)]) # , c="blue"

        plt.title(f"Z-score - t-test {sector1} {sector2}")
        plt.xlabel("Z-score")
        plt.ylabel("t-test")

        plt.savefig(
            f"./results/economic_results/o_info/images/divided_years/gaussian_before/statistical_significance/scatterplot_Z_score_t_test_sector_{sector1}_{sector2}.pdf",
            dpi=600,
            bbox_inches="tight",
        )

        plt.show()

In [ ]:
for sector1 in sectors:
    fig, axs = plt.subplots(2, 4, figsize=(20, 10))

    for i, sector2 in enumerate(sectors):
        df_z = pd.read_csv(
            f"./results/economic_results/o_info/dataframes/divided_years/gaussian_before/statistical_significance/dataframe_z_score_{sector1}_{sector2}.csv",
            index_col=0,
        )
        df_t = pd.read_csv(
            f"./results/economic_results/o_info/dataframes/divided_years/gaussian_before/statistical_significance/dataframe_t_values_{sector1}_{sector2}.csv",
            index_col=0,
        )

        row = i // 4
        col = i % 4

        for year in years:
            axs[row, col].scatter(df_z[str(year)], df_t[str(year)]) # , c="blue"

        axs[row, col].set_title(f"{sector2}", size=17)
        axs[row, col].set_xlabel("Z-score", size=14)
        axs[row, col].set_ylabel("t-test", size=14)

    fig.suptitle(f"{sector1}", fontsize=30, y=1.01)
    plt.tight_layout()
    
    plt.savefig(
        f"./results/economic_results/o_info/images/divided_years/gaussian_before/statistical_significance/scatterplot_Z_score_t_test_sector_{sector1}.pdf",
        dpi=600,
        bbox_inches="tight",
    )
    plt.show()

In [ ]:
for sector1 in sectors:
    fig, axs = plt.subplots(2, 4, figsize=(20, 10))

    for i, sector2 in enumerate(sectors):
        df_z = pd.read_csv(
            f"./results/economic_results/o_info/dataframes/divided_years/gaussian_before/statistical_significance/dataframe_z_p_value_{sector1}_{sector2}.csv",
            index_col=0,
        )
        df_t = pd.read_csv(
            f"./results/economic_results/o_info/dataframes/divided_years/gaussian_before/statistical_significance/dataframe_t_p_values_{sector1}_{sector2}.csv",
            index_col=0,
        )

        row = i // 4
        col = i % 4

        for year in years:
            axs[row, col].scatter(df_z[str(year)], df_t[str(year)], c="blue")
        axs[row, col].axhline(y=0.05, color="r", linestyle="--")

        axs[row, col].set_title(f"{sector2}", size=17)
        axs[row, col].set_xlabel("p-value Z-score", size=14)
        axs[row, col].set_ylabel("p-value t-test", size=14)

    fig.suptitle(f"{sector1}", fontsize=30, y=1.01)
    plt.tight_layout()
    
    plt.savefig(
       f"./results/economic_results/o_info/images/divided_years/gaussian_before/statistical_significance/scatterplot_p_values_Z_score_t_test_sector_{sector1}.pdf",
       dpi=600,
       bbox_inches="tight",
    )
    plt.show()

<a name="o_info_one_year_null_model_divided_randomised"></a>
##### Divided by year and randomised

In [ ]:
financial_ts["Date"] = pd.to_datetime(financial_ts["Date"])
years = financial_ts["Date"].dt.year.unique()[1:]

diz_one_year = {}

for year in years:
    df_year = financial_ts[financial_ts["Date"].dt.year == year].copy()
    df_year["Date"] = pd.to_datetime(df_year["Date"])
    diz_one_year[year] = df_year

In [ ]:
diz_one_year_gauss = {}

for year in years:
    average_columns = diz_one_year[year].iloc[:, 1:].mean()
    covariance = diz_one_year[year].iloc[:, 1:].cov()
    num_samples = len(diz_one_year[year])

    generated_values = np.random.multivariate_normal(
        average_columns, covariance, num_samples
    )
    df_gauss = pd.DataFrame(
        generated_values, columns=diz_one_year[year].iloc[:, 1:].columns
    )

    first_column = diz_one_year[year].iloc[:, 0]
    df_gauss.insert(0, "Date", first_column)

    diz_one_year_gauss[year] = df_gauss

In [ ]:
tw = 7
diz_one_year_gaussian_bin = {}

for year in years:
    diz_one_year_gaussian_bin[year] = binarize_df(diz_one_year_gauss[year], tw)

In [ ]:
# DO NOT RUN IF YOU ALREADY HAVE THE DATA

for year in years:
    for target in sectors:
        oinfo_one_sector = get_o_infos_same(
            target, diz_one_year_gaussian_bin[year], total=True
        )
        np.save(
            f"./results/economic_results/o_info/metrics/3/divided_years/gaussian_after/one_sector_{target}_year_{year}.npy",
            oinfo_one_sector,
        )

for year in years:
    for target in sectors:
        other_sectors = [sector for sector in sectors if sector != target]
        for i, sector in enumerate(other_sectors):

            oinfo_two_sectors = get_o_infos_two_different(
                target, sector, diz_one_year_gaussian_bin[year], total=True
            )
            np.save(
                f"./results/economic_results/o_info/metrics/2_1/divided_years/gaussian_after/one_sector_{target}_two_sector_{sector}_year_{year}.npy",
                oinfo_two_sectors,
            )

for year in years:
    for target in sectors:
        other_sectors = [sector for sector in sectors if sector != target]
        for sector1 in other_sectors:
            other_sectors2 = [sector for sector in other_sectors if sector != sector1]
            for sector2 in other_sectors2:

                oinfo_three_sectors = get_o_infos_three_different(
                    target,
                    sector1,
                    sector2,
                    diz_one_year_gaussian_bin[year],
                    total=True,
                )
                np.save(
                    f"./results/economic_results/o_info/metrics/1_1_1/divided_years/gaussian_after/one_{target}_one_{sector1}_one_{sector2}_year_{year}.npy",
                    oinfo_three_sectors,
                )

In [ ]:
# DO NOT RUN IF YOU ALREADY HAVE THE DATAFRAMES

for target1 in sectors:
    for target2 in sectors:
        df_one_year = pd.DataFrame(columns=years)
        for year in years:
            for source in sectors:

                if target2 == target1 and source == target1:
                    oinfo_one_sector = np.load(
                        f"./results/economic_results/o_info/metrics/3/divided_years/gaussian_after/one_sector_{target2}_year_{year}.npy"
                    )
                    df_one_year.at[source, year] = np.mean(oinfo_one_sector)

                elif target2 == target1 and source != target1:
                    oinfo_two_sectors = np.load(
                        f"./results/economic_results/o_info/metrics/2_1/divided_years/gaussian_after/one_sector_{source}_two_sector_{target2}_year_{year}.npy"
                    )
                    df_one_year.at[source, year] = np.mean(oinfo_two_sectors)

                elif target2 != target1 and source == target1:
                    oinfo_two_sectors = np.load(
                        f"./results/economic_results/o_info/metrics/2_1/divided_years/gaussian_after/one_sector_{target2}_two_sector_{source}_year_{year}.npy"
                    )
                    df_one_year.at[source, year] = np.mean(oinfo_two_sectors)

                elif target2 != target1 and source == target2:
                    oinfo_two_sectors = np.load(
                        f"./results/economic_results/o_info/metrics/2_1/divided_years/gaussian_after/one_sector_{target1}_two_sector_{target2}_year_{year}.npy"
                    )
                    df_one_year.at[source, year] = np.mean(oinfo_two_sectors)

                elif target2 != target1 and source != target2:
                    oinfo_three_sectors = np.load(
                        f"./results/economic_results/o_info/metrics/1_1_1/divided_years/gaussian_after/one_{target1}_one_{target2}_one_{source}_year_{year}.npy"
                    )
                    df_one_year.at[source, year] = np.mean(oinfo_three_sectors)

        df_one_year.to_csv(
            f"./results/economic_results/o_info/dataframes/divided_years/gaussian_after/dataframe_{target1}_{target2}.csv",
            index=True,
        )

In [ ]:
for target1 in sectors:
    for target2 in sectors:

        df = pd.read_csv(
            f"./results/economic_results/o_info/dataframes/divided_years/gaussian_after/dataframe_{target1}_{target2}.csv",
            index_col=0,
        )

        sns.heatmap(df, cmap="Blues")

        plt.axvline(x=1.5, color="brown", linestyle="--", label="Twin towers")
        plt.axvline(x=8.5, color="blue", linestyle="--", label="Financial crisis")
        plt.axvline(x=9.5, color="green", linestyle="--", label="Obama president")
        plt.axvline(
            x=11.5,
            color="yellow",
            linestyle="--",
            label="European sovereign debt crisis / Quantitative easing USA",
        )
        plt.axvline(x=17.5, color="darkorchid", linestyle="--", label="Trump president")
        plt.axvline(x=19.5, color="magenta", linestyle="--", label="Covid-19")
        plt.axvline(x=21.5, color="teal", linestyle="--", label="Biden president")

        plt.xlabel("Year", fontsize=15)
        plt.ylabel("Source", fontsize=15)
        plt.title(f"After split: 1 {target1}, 1 {target2}", fontsize=18)

        # plt.legend()

        plt.savefig(
            f"./results/economic_results/o_info/images/divided_years/gaussian_after/heatmap_{target1}_{target2}.pdf",
            dpi=600,
            bbox_inches="tight",
        )

        plt.show()

In [ ]:
mini = [50 for i in range(len(sectors))]
maxi = [-50 for i in range(len(sectors))]

for i, target1 in enumerate(sectors):
    for target2 in sectors:
        df = pd.read_csv(
            f"./results/economic_results/o_info/dataframes/divided_years/gaussian_after/dataframe_{target1}_{target2}.csv",
            index_col=0,
        )
        if min(df.min()) < mini[i]:
            mini[i] = min(df.min())
        if max(df.max()) > maxi[i]:
            maxi[i] = max(df.max())

for index1, sector1 in enumerate(sectors):
    fig, axs = plt.subplots(2, 4, figsize=(21, 12))

    for index2, sector2 in enumerate(sectors):
        df = pd.read_csv(
            f"./results/economic_results/o_info/dataframes/divided_years/gaussian_after/dataframe_{sector1}_{sector2}.csv",
            index_col=0,
        )

        row = index2 // 4
        col = index2 % 4

        ax = axs[row, col]

        sns.heatmap(df, ax=ax, cmap="Blues", vmin=mini[index1], vmax=maxi[index1])

        ax.axvline(x=1.5, color="brown", linestyle="--", label="Twin towers")
        ax.axvline(x=8.5, color="blue", linestyle="--", label="Financial crisis")
        ax.axvline(x=9.5, color="green", linestyle="--", label="Obama president")
        ax.axvline(
            x=11.5,
            color="yellow",
            linestyle="--",
            label="European sovereign debt crisis / Quantitative easing USA",
        )
        ax.axvline(x=17.5, color="darkorchid", linestyle="--", label="Trump president")
        ax.axvline(x=19.5, color="magenta", linestyle="--", label="Covid-19")
        ax.axvline(x=21.5, color="teal", linestyle="--", label="Biden president")

        ax.set_title(f"{sector2}", size=25)
        ax.set_xticks(
            [0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20],
            labels=[
                "2000",
                "2002",
                "2004",
                "2006",
                "2008",
                "2010",
                "2012",
                "2014",
                "2016",
                "2018",
                "2020",
            ],
        )
        ax.set_yticklabels([])

    fig.text(0.485, -0.03, "Year", ha="center", va="center", fontsize=28)
    fig.text(-0.02, 0.48, "Sector", rotation="vertical", va="center", fontsize=28)

    plt.suptitle(f"After split: {sector1}", size=40, x=0.485)
    plt.tight_layout()

    plt.savefig(
        f"./results/economic_results/o_info/images/divided_years/gaussian_after/heatmap_grouped_{sector1}.pdf",
        dpi=600,
        bbox_inches="tight",
    )

    plt.show()

In [ ]:
mini_gauss = [50 for i in range(len(sectors))]
maxi_gauss = [-50 for i in range(len(sectors))]

for i, target1 in enumerate(sectors):
    for target2 in sectors:
        df = pd.read_csv(
            f"./results/economic_results/o_info/dataframes/divided_years/gaussian_after/dataframe_{target1}_{target2}.csv",
            index_col=0,
        )
        if min(df.min()) < mini_gauss[i]:
            mini_gauss[i] = min(df.min())
        if max(df.max()) > maxi_gauss[i]:
            maxi_gauss[i] = max(df.max())

mini_original = [50 for i in range(len(sectors))]
maxi_original = [-50 for i in range(len(sectors))]

for i, target1 in enumerate(sectors):
    for target2 in sectors:
        df = pd.read_csv(
            f"./results/economic_results/o_info/dataframes/divided_years/dataframe_{target1}_{target2}.csv",
            index_col=0,
        )
        if min(df.min()) < mini_original[i]:
            mini_original[i] = min(df.min())
        if max(df.max()) > maxi_original[i]:
            maxi_original[i] = max(df.max())


for index1, sector1 in enumerate(sectors):
    fig, axs = plt.subplots(2, 8, figsize=(40, 12))

    for index2, sector2 in enumerate(sectors):
        col = index2 % 8

        ax = axs[0, col]
        df = pd.read_csv(
            f"./results/economic_results/o_info/dataframes/divided_years/dataframe_{sector1}_{sector2}.csv",
            index_col=0,
        )
        sns.heatmap(
            df,
            ax=ax,
            cmap="Blues",
            vmin=mini_original[index1],
            vmax=maxi_original[index1],
        )
        ax.axvline(x=1.5, color="brown", linestyle="--", label="Twin towers")
        ax.axvline(x=8.5, color="blue", linestyle="--", label="Financial crisis")
        ax.axvline(x=9.5, color="green", linestyle="--", label="Obama president")
        ax.axvline(
            x=11.5,
            color="yellow",
            linestyle="--",
            label="European sovereign debt crisis / Quantitative easing USA",
        )
        ax.axvline(x=17.5, color="darkorchid", linestyle="--", label="Trump president")
        ax.axvline(x=19.5, color="magenta", linestyle="--", label="Covid-19")
        ax.axvline(x=21.5, color="teal", linestyle="--", label="Biden president")

        ax.set_title(f"{sector2} (original)", size=22)
        ax.set_xticks(
            [0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20],
            labels=[
                "2000",
                "2002",
                "2004",
                "2006",
                "2008",
                "2010",
                "2012",
                "2014",
                "2016",
                "2018",
                "2020",
            ],
        )
        ax.set_yticklabels([])

        ax = axs[1, col]
        df = pd.read_csv(
            f"./results/economic_results/o_info/dataframes/divided_years/gaussian_after/dataframe_{sector1}_{sector2}.csv",
            index_col=0,
        )
        sns.heatmap(
            df, ax=ax, cmap="Blues", vmin=mini_gauss[index1], vmax=maxi_gauss[index1]
        )

        ax.axvline(x=1.5, color="brown", linestyle="--", label="Twin towers")
        ax.axvline(x=8.5, color="blue", linestyle="--", label="Financial crisis")
        ax.axvline(x=9.5, color="green", linestyle="--", label="Obama president")
        ax.axvline(
            x=11.5,
            color="yellow",
            linestyle="--",
            label="European sovereign debt crisis / Quantitative easing USA",
        )
        ax.axvline(x=17.5, color="darkorchid", linestyle="--", label="Trump president")
        ax.axvline(x=19.5, color="magenta", linestyle="--", label="Covid-19")
        ax.axvline(x=21.5, color="teal", linestyle="--", label="Biden president")

        ax.set_title(f"{sector2} (null)", size=22)
        ax.set_xticks(
            [0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20],
            labels=[
                "2000",
                "2002",
                "2004",
                "2006",
                "2008",
                "2010",
                "2012",
                "2014",
                "2016",
                "2018",
                "2020",
            ],
        )
        ax.set_yticklabels([])

    fig.text(0.49, -0.03, "Year", ha="center", va="center", fontsize=30)
    fig.text(-0.015, 0.465, "Sector", rotation="vertical", va="center", fontsize=30)

    plt.suptitle(f"Comparison after split: {sector1}", size=40, x=0.49)
    plt.tight_layout()

    plt.savefig(
        f"./results/economic_results/o_info/images/divided_years/gaussian_after/heatmap_grouped_comparison_original_gaussian_after_{sector1}.pdf",
        dpi=600,
        bbox_inches="tight",
    )

    plt.show()

<a name="o_info_one_year_null_model_divided_randomised_stat"></a>
###### Statistical significance

<a name="o_info_one_year_null_model_divided_randomised_stat_t_test"></a>
###### t-test

In [ ]:
financial_ts["Date"] = pd.to_datetime(financial_ts["Date"])
years = financial_ts["Date"].dt.year.unique()[1:]

In [ ]:
# DO NOT RUN IF YOU ALREADY HAVE THE DATAFRAMES

for target1 in sectors:
    for target2 in sectors:
        df_one_year_t_value = pd.DataFrame(columns=years)
        df_one_year_p_value = pd.DataFrame(columns=years)
        for year in years:
            for source in sectors:

                if target2 == target1 and source == target1:
                    oinfo_one_sector_original = np.load(
                        f"./results/economic_results/o_info/metrics/3/divided_years/one_sector_{target2}_year_{year}.npy"
                    )
                    oinfo_one_sector_gauss = np.load(
                        f"./results/economic_results/o_info/metrics/3/divided_years/gaussian_after/one_sector_{target2}_year_{year}.npy"
                    )
                    t, p = t_test(oinfo_one_sector_original, oinfo_one_sector_gauss)
                    df_one_year_t_value.at[source, year] = t
                    df_one_year_p_value.at[source, year] = p

                elif target2 == target1 and source != target1:
                    oinfo_two_sectors_original = np.load(
                        f"./results/economic_results/o_info/metrics/2_1/divided_years/one_sector_{source}_two_sector_{target2}_year_{year}.npy"
                    )
                    oinfo_two_sectors_gauss = np.load(
                        f"./results/economic_results/o_info/metrics/2_1/divided_years/gaussian_after/one_sector_{source}_two_sector_{target2}_year_{year}.npy"
                    )
                    t, p = t_test(oinfo_two_sectors_original, oinfo_two_sectors_gauss)
                    df_one_year_t_value.at[source, year] = t
                    df_one_year_p_value.at[source, year] = p

                elif target2 != target1 and source == target1:
                    oinfo_two_sectors_original = np.load(
                        f"./results/economic_results/o_info/metrics/2_1/divided_years/one_sector_{target2}_two_sector_{source}_year_{year}.npy"
                    )
                    oinfo_two_sectors_gauss = np.load(
                        f"./results/economic_results/o_info/metrics/2_1/divided_years/gaussian_after/one_sector_{target2}_two_sector_{source}_year_{year}.npy"
                    )
                    t, p = t_test(oinfo_two_sectors_original, oinfo_two_sectors_gauss)
                    df_one_year_t_value.at[source, year] = t
                    df_one_year_p_value.at[source, year] = p

                elif target2 != target1 and source == target2:
                    oinfo_two_sectors_original = np.load(
                        f"./results/economic_results/o_info/metrics/2_1/divided_years/one_sector_{target1}_two_sector_{target2}_year_{year}.npy"
                    )
                    oinfo_two_sectors_gauss = np.load(
                        f"./results/economic_results/o_info/metrics/2_1/divided_years/gaussian_after/one_sector_{target1}_two_sector_{target2}_year_{year}.npy"
                    )
                    t, p = t_test(oinfo_two_sectors_original, oinfo_two_sectors_gauss)
                    df_one_year_t_value.at[source, year] = t
                    df_one_year_p_value.at[source, year] = p

                elif target2 != target1 and source != target2:
                    oinfo_three_sectors_original = np.load(
                        f"./results/economic_results/o_info/metrics/1_1_1/divided_years/one_sector_{target1}_one_sector_{target2}_one_sector_{source}_year_{year}.npy"
                    )
                    oinfo_three_sectors_gauss = np.load(
                        f"./results/economic_results/o_info/metrics/1_1_1/divided_years/gaussian_after/one_{target1}_one_{target2}_one_{source}_year_{year}.npy"
                    )
                    t, p = t_test(
                        oinfo_three_sectors_original, oinfo_three_sectors_gauss
                    )
                    df_one_year_t_value.at[source, year] = t
                    df_one_year_p_value.at[source, year] = p

        df_one_year_t_value.to_csv(
            f"./results/economic_results/o_info/dataframes/divided_years/gaussian_after/statistical_significance/dataframe_t_values_{target1}_{target2}.csv",
            index=True,
        )
        df_one_year_p_value.to_csv(
            f"./results/economic_results/o_info/dataframes/divided_years/gaussian_after/statistical_significance/dataframe_t_p_values_{target1}_{target2}.csv",
            index=True,
        )

In [ ]:
for index1, sector1 in enumerate(sectors):
    fig, axs = plt.subplots(2, 4, figsize=(21, 12))

    for index2, sector2 in enumerate(sectors):
        row = index2 // 4
        col = index2 % 4
        ax = axs[row, col]

        df_t = pd.read_csv(
            f"./results/economic_results/o_info/dataframes/divided_years/gaussian_after/statistical_significance/dataframe_t_values_{sector1}_{sector2}.csv",
            index_col=0,
        )
        df_t = df_t.applymap(sign_t)
        df_p = pd.read_csv(
            f"./results/economic_results/o_info/dataframes/divided_years/gaussian_after/statistical_significance/dataframe_t_p_values_{sector1}_{sector2}.csv",
            index_col=0,
        )
        df_p = df_p.applymap(sign_p)

        cmap = sns.color_palette(["lightblue"])

        force_break_loop = False
        for column in df_p.columns:
            for index, row in df_p.iterrows():
                if row[column] == "red":
                    cmap = sns.color_palette(["white", "lightblue"])
                    force_break_loop = True
                    break
            if force_break_loop:
                break

        cmap = sns.color_palette(["white", "lightblue"])
        sns.heatmap(
            df_p.replace({"red": 0, "green": 1}),
            cmap=cmap,
            annot=df_t,
            fmt="s",
            cbar=False,
            ax=ax,
        )

        ax.axvline(x=1.5, color="brown", linestyle="--", label="Twin towers")
        ax.axvline(x=8.5, color="blue", linestyle="--", label="Financial crisis")
        ax.axvline(x=9.5, color="green", linestyle="--", label="Obama president")
        ax.axvline(
            x=11.5,
            color="yellow",
            linestyle="--",
            label="European sovereign debt crisis / Quantitative easing USA",
        )
        ax.axvline(x=17.5, color="darkorchid", linestyle="--", label="Trump president")
        ax.axvline(x=19.5, color="magenta", linestyle="--", label="Covid-19")
        ax.axvline(x=21.5, color="teal", linestyle="--", label="Biden president")

        ax.set_title(f"{sector2} (t-test)", size=22)
        ax.set_xticks(
            [0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20],
            labels=[
                "2000",
                "2002",
                "2004",
                "2006",
                "2008",
                "2010",
                "2012",
                "2014",
                "2016",
                "2018",
                "2020",
            ],
        )
        ax.set_yticklabels([])

    fig.text(0.50, -0.03, "Year", ha="center", va="center", fontsize=25)
    fig.text(-0.02, 0.472, "Sector", rotation="vertical", va="center", fontsize=25)

    plt.suptitle(f"After split: {sector1}", size=40, x=0.50)
    plt.tight_layout()

    plt.savefig(
        f"./results/economic_results/o_info/images/divided_years/gaussian_after/statistical_significance/heatmap_grouped_after_stat_sign_t_test_{sector1}.pdf",
        dpi=600,
        bbox_inches="tight",
    )

    plt.show()

<a name="o_info_one_year_null_model_divided_randomised_stat_Z_score"></a>
###### Z-score

In [ ]:
financial_ts["Date"] = pd.to_datetime(financial_ts["Date"])
years = financial_ts["Date"].dt.year.unique()[1:]

In [ ]:
# DO NOT RUN IF YOU ALREADY HAVE THE DATAFRAMES

for target1 in sectors:
    for target2 in sectors:
        df_one_year_z_score = pd.DataFrame(columns=years)
        df_one_year_p_value = pd.DataFrame(columns=years)
        for year in years:
            for source in sectors:

                if target2 == target1 and source == target1:
                    oinfo_one_sector_original = np.load(
                        f"./results/economic_results/o_info/metrics/3/divided_years/one_sector_{target2}_year_{year}.npy"
                    )
                    oinfo_one_sector_gauss = np.load(
                        f"./results/economic_results/o_info/metrics/3/divided_years/gaussian_after/one_sector_{target2}_year_{year}.npy"
                    )
                    z, p = z_score(oinfo_one_sector_original, oinfo_one_sector_gauss)
                    df_one_year_z_score.at[source, year] = z
                    df_one_year_p_value.at[source, year] = p

                elif target2 == target1 and source != target1:
                    oinfo_two_sectors_original = np.load(
                        f"./results/economic_results/o_info/metrics/2_1/divided_years/one_sector_{source}_two_sector_{target2}_year_{year}.npy"
                    )
                    oinfo_two_sectors_gauss = np.load(
                        f"./results/economic_results/o_info/metrics/2_1/divided_years/gaussian_after/one_sector_{source}_two_sector_{target2}_year_{year}.npy"
                    )
                    z, p = z_score(oinfo_two_sectors_original, oinfo_two_sectors_gauss)
                    df_one_year_z_score.at[source, year] = z
                    df_one_year_p_value.at[source, year] = p

                elif target2 != target1 and source == target1:
                    oinfo_two_sectors_original = np.load(
                        f"./results/economic_results/o_info/metrics/2_1/divided_years/one_sector_{target2}_two_sector_{source}_year_{year}.npy"
                    )
                    oinfo_two_sectors_gauss = np.load(
                        f"./results/economic_results/o_info/metrics/2_1/divided_years/gaussian_after/one_sector_{target2}_two_sector_{source}_year_{year}.npy"
                    )
                    z, p = z_score(oinfo_two_sectors_original, oinfo_two_sectors_gauss)
                    df_one_year_z_score.at[source, year] = z
                    df_one_year_p_value.at[source, year] = p

                elif target2 != target1 and source == target2:
                    oinfo_two_sectors_original = np.load(
                        f"./results/economic_results/o_info/metrics/2_1/divided_years/one_sector_{target1}_two_sector_{target2}_year_{year}.npy"
                    )
                    oinfo_two_sectors_gauss = np.load(
                        f"./results/economic_results/o_info/metrics/2_1/divided_years/gaussian_after/one_sector_{target1}_two_sector_{target2}_year_{year}.npy"
                    )
                    z, p = z_score(oinfo_two_sectors_original, oinfo_two_sectors_gauss)
                    df_one_year_z_score.at[source, year] = z
                    df_one_year_p_value.at[source, year] = p

                elif target2 != target1 and source != target2:
                    oinfo_three_sectors_original = np.load(
                        f"./results/economic_results/o_info/metrics/1_1_1/divided_years/one_sector_{target1}_one_sector_{target2}_one_sector_{source}_year_{year}.npy"
                    )
                    oinfo_three_sectors_gauss = np.load(
                        f"./results/economic_results/o_info/metrics/1_1_1/divided_years/gaussian_after/one_{target1}_one_{target2}_one_{source}_year_{year}.npy"
                    )
                    z, p = z_score(
                        oinfo_three_sectors_original, oinfo_three_sectors_gauss
                    )
                    df_one_year_z_score.at[source, year] = z
                    df_one_year_p_value.at[source, year] = p

        df_one_year_z_score.to_csv(
            f"./results/economic_results/o_info/dataframes/divided_years/gaussian_after/statistical_significance/dataframe_z_score_{target1}_{target2}.csv",
            index=True,
        )
        df_one_year_p_value.to_csv(
            f"./results/economic_results/o_info/dataframes/divided_years/gaussian_after/statistical_significance/dataframe_z_p_value_{target1}_{target2}.csv",
            index=True,
        )

In [ ]:
mini = [50 for i in range(len(sectors))]
maxi = [-50 for i in range(len(sectors))]

for i, target1 in enumerate(sectors):
    for target2 in sectors:
        df = pd.read_csv(
            f"./results/economic_results/o_info/dataframes/divided_years/gaussian_after/statistical_significance/dataframe_z_score_{target1}_{target2}.csv",
            index_col=0,
        )
        if min(df.min()) < mini[i]:
            mini[i] = min(df.min())
        if max(df.max()) > maxi[i]:
            maxi[i] = max(df.max())

for index1, sector1 in enumerate(sectors):
    fig, axs = plt.subplots(2, 4, figsize=(21, 12))

    for index2, sector2 in enumerate(sectors):
        df_z = pd.read_csv(
            f"./results/economic_results/o_info/dataframes/divided_years/gaussian_after/statistical_significance/dataframe_z_score_{sector1}_{sector2}.csv",
            index_col=0,
        )

        row = index2 // 4
        col = index2 % 4

        ax = axs[row, col]

        sns.heatmap(
            df_z, ax=ax, cmap="coolwarm", vmin=mini[index1], vmax=maxi[index1]
        )

        ax.axvline(x=1.5, color="brown", linestyle="--", label="Twin towers")
        ax.axvline(x=8.5, color="blue", linestyle="--", label="Financial crisis")
        ax.axvline(x=9.5, color="green", linestyle="--", label="Obama president")
        ax.axvline(
            x=11.5,
            color="yellow",
            linestyle="--",
            label="European sovereign debt crisis / Quantitative easing USA",
        )
        ax.axvline(x=17.5, color="darkorchid", linestyle="--", label="Trump president")
        ax.axvline(x=19.5, color="magenta", linestyle="--", label="Covid-19")
        ax.axvline(x=21.5, color="teal", linestyle="--", label="Biden president")

        ax.set_title(f"{sector2} (Z-score)", size=22)
        ax.set_xticks(
            [0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20],
            labels=[
                "2000",
                "2002",
                "2004",
                "2006",
                "2008",
                "2010",
                "2012",
                "2014",
                "2016",
                "2018",
                "2020",
            ],
        )
        ax.set_yticklabels([])

    fig.text(0.485, -0.03, "Year", ha="center", va="center", fontsize=25)
    fig.text(-0.02, 0.478, "Sector", rotation="vertical", va="center", fontsize=25)

    plt.suptitle(f"After split: {sector1}", size=40, x=0.485)
    plt.tight_layout()

    plt.savefig(
        f"./results/economic_results/o_info/images/divided_years/gaussian_after/statistical_significance/heatmap_grouped_after_stat_sign_Z_score_{sector1}.pdf",
        dpi=600,
        bbox_inches="tight",
    )

    plt.show()

<a name="o_info_one_year_null_model_divided_randomised_stat_comparison"></a>
###### Comparison

In [ ]:
financial_ts["Date"] = pd.to_datetime(financial_ts["Date"])
years = financial_ts["Date"].dt.year.unique()[1:]

In [ ]:
for sector1 in sectors:
    for sector2 in sectors:
        plt.figure(figsize=(6, 4))

        df_z = pd.read_csv(
            f"./results/economic_results/o_info/dataframes/divided_years/gaussian_after/statistical_significance/dataframe_z_score_{sector1}_{sector2}.csv",
            index_col=0,
        )
        df_t = pd.read_csv(
            f"./results/economic_results/o_info/dataframes/divided_years/gaussian_after/statistical_significance/dataframe_t_values_{sector1}_{sector2}.csv",
            index_col=0,
        )

        for year in years:
            plt.scatter(df_z[str(year)], df_t[str(year)]) # , c="blue"

        plt.title(f"Z-score - t-test {sector1} {sector2}")
        plt.xlabel("Z-score")
        plt.ylabel("t-test")

        plt.savefig(
            f"./results/economic_results/o_info/images/divided_years/gaussian_after/statistical_significance/scatterplot_Z_score_t_test_sector_{sector1}_{sector2}.pdf",
            dpi=600,
            bbox_inches="tight",
        )

        plt.show()

In [ ]:
for sector1 in sectors:
    fig, axs = plt.subplots(2, 4, figsize=(20, 10))

    for i, sector2 in enumerate(sectors):
        df_z = pd.read_csv(
            f"./results/economic_results/o_info/dataframes/divided_years/gaussian_after/statistical_significance/dataframe_z_score_{sector1}_{sector2}.csv",
            index_col=0,
        )
        df_t = pd.read_csv(
            f"./results/economic_results/o_info/dataframes/divided_years/gaussian_after/statistical_significance/dataframe_t_values_{sector1}_{sector2}.csv",
            index_col=0,
        )

        row = i // 4
        col = i % 4

        for year in years:
            axs[row, col].scatter(df_z[str(year)], df_t[str(year)]) # , c="blue"

        axs[row, col].set_title(f"{sector2}", size=17)
        axs[row, col].set_xlabel("Z-score", size=14)
        axs[row, col].set_ylabel("t-test", size=14)

    fig.suptitle(f"{sector1}", fontsize=30, y=1.01)
    plt.tight_layout()
    
    plt.savefig(
        f"./results/economic_results/o_info/images/divided_years/gaussian_after/statistical_significance/scatterplot_Z_score_t_test_sector_{sector1}.pdf",
        dpi=600,
        bbox_inches="tight",
    )
    plt.show()

In [ ]:
for sector1 in sectors:
    fig, axs = plt.subplots(2, 4, figsize=(20, 10))

    for i, sector2 in enumerate(sectors):
        df_z = pd.read_csv(
            f"./results/economic_results/o_info/dataframes/divided_years/gaussian_after/statistical_significance/dataframe_z_p_value_{sector1}_{sector2}.csv",
            index_col=0,
        )
        df_t = pd.read_csv(
            f"./results/economic_results/o_info/dataframes/divided_years/gaussian_after/statistical_significance/dataframe_t_p_values_{sector1}_{sector2}.csv",
            index_col=0,
        )

        row = i // 4
        col = i % 4

        for year in years:
            axs[row, col].scatter(df_z[str(year)], df_t[str(year)], c="blue")
        axs[row, col].axhline(y=0.05, color="r", linestyle="--")

        axs[row, col].set_title(f"{sector2}", size=17)
        axs[row, col].set_xlabel("p-value Z-score", size=14)
        axs[row, col].set_ylabel("p-value t-test", size=14)

    fig.suptitle(f"{sector1}", fontsize=30, y=1.01)
    plt.tight_layout()
    
    plt.savefig(
       f"./results/economic_results/o_info/images/divided_years/gaussian_after/statistical_significance/scatterplot_p_values_Z_score_t_test_sector_{sector1}.pdf",
       dpi=600,
       bbox_inches="tight",
    )
    plt.show()

<a name="o_info_one_year_evolution_time"></a>
#### Evolution in time

In [ ]:
financial_ts["Date"] = pd.to_datetime(financial_ts["Date"])
years = financial_ts["Date"].dt.year.unique()[1:]

In [ ]:
# All in the same sector
df_same = pd.DataFrame(columns=[str(year) for year in years])

for sector in sectors:
    df = pd.read_csv(
        f"./results/economic_results/o_info/dataframes/divided_years/dataframe_{sector}_{sector}.csv",
        index_col=0,
    )
    df_same.loc[sector] = df.loc[sector]

In [ ]:
# Not all in the same sector
diz_not_same = {}

for sector in sectors:
    columns = [str(year) for year in years]
    columns = ["Sector"] + columns
    df_not_same = pd.DataFrame(columns=columns)

    row = 0

    # 2 in sector 1 not
    df = pd.read_csv(
        f"./results/economic_results/o_info/dataframes/divided_years/dataframe_{sector}_{sector}.csv",
        index_col=0,
    )
    other_sectors = [sector2 for sector2 in sectors if sector2 != sector]
    for sector2 in other_sectors:
        df_not_same.loc[row] = [sector2] + df.loc[sector2].astype(float).tolist()
        row += 1

    # 1 in sector and 2 not
    for sector2 in other_sectors:
        df = pd.read_csv(
            f"./results/economic_results/o_info/dataframes/divided_years/dataframe_{sector}_{sector2}.csv",
            index_col=0,
        )
        df_not_same.loc[row] = [sector2] + df.loc[sector2].astype(float).tolist()
        row += 1

    df_mean = df_not_same.groupby("Sector").mean()
    df_mean = df_mean.reindex(other_sectors)
    
    diz_not_same[sector] = df_mean

In [ ]:
df_transposed = df_same.transpose()

for column in df_transposed:
    plt.plot(df_transposed.index, df_transposed[column], label=column)

plt.xticks([str(year) for year in range(2000, 2022, 3)])
    
plt.legend(fontsize="small")
plt.title("3 in the same sector", size=17)
plt.show()

In [ ]:
fig, axs = plt.subplots(2, 4, figsize=(15, 8))

for index, sector in enumerate(sectors):
    row = index // 4
    col = index % 4
    ax = axs[row, col]

    df_not_same = diz_not_same[sector]
    df_transposed = df_not_same.transpose()

    for column in df_transposed:
        ax.plot(df_transposed.index, df_transposed[column], label=column)

    ax.set_xticks([str(year) for year in range(2000, 2022, 5)])

    ax.legend(fontsize="x-small")
    ax.set_title(f"1 or 2 in {sector}", size=15)

fig.text(0.515, -0.02, "Years", ha="center", va="center", fontsize=20)
fig.text(-0.015, 0.5, "O-information", rotation="vertical", va="center", fontsize=20)

plt.tight_layout()

plt.show()